In [5]:
import os
import gc
# import cv2
from PIL import Image
import numpy as np

# 加载所有图片，共64467张


def load_data():
    # Return a new array of given shape and type, without initializing entries.
    data = np.empty((64467, 60, 80, 3), dtype='float32')
#     data_arr=[[[[]]]]
#     label = np.empty((64467, 16), dtype='string')
    label_arr = [["" for i in range(16)]for j in range(64467)]
    # os.listdir(filename)返回filename中所有文件的文件名列表
    imgs = os.listdir('cardata')
    num = len(imgs)
    print(num)
    for i in range(num):
        # PIL 的 open() 函数用于创建 PIL 图像对象
        img_0 = Image.open('cardata/'+imgs[i])
        img = img_0.resize((80, 60), Image.ANTIALIAS)
        # Convert the input to an array
        arr = np.asarray(img, dtype='float32')
        data[i, :, :, :] = arr

        label_arr[i] = imgs[i].split('_')[0:16]  # 这个是字符串类型需要编码
        if i % 100 == 0:
            print(i/646, "%")
        del arr
        del img
        del img_0
        gc.collect()
    return data, label_arr

# 加载图片和标签

在初始化data的时候，使用data = np.empty((64000, 240, 320, 3), dtype='float32')会导致内存不足。针对此问题，首先采用的解决方案是先声明一个四维数组arr，每读一张图就往数组arr里头写入一个三维数组img。最后把数组转换为numpyarray，实在转换不了就算了

经实验，上述方法不能解决问题，现在启用plan B，换数据集。换数据集不光是出于内存的考虑，而是当前数据集中没有国产车。最后的训练结果一定是不尽人意的

plan C 减小图片像素，利用此数据集进行训练。之后手动标注国产车进行迁移学习。

构造label的时候，由于有些标签是字符串类型，所以需要先存储到一个多维数组里，最后进行编码处理

In [6]:
data, label_arr = load_data()


64467
0.0 %
0.15479876160990713 %
0.30959752321981426 %
0.46439628482972134 %
0.6191950464396285 %
0.7739938080495357 %
0.9287925696594427 %
1.08359133126935 %
1.238390092879257 %
1.3931888544891642 %
1.5479876160990713 %
1.7027863777089782 %
1.8575851393188854 %
2.0123839009287927 %
2.1671826625387 %
2.321981424148607 %
2.476780185758514 %
2.6315789473684212 %
2.7863777089783284 %
2.9411764705882355 %
3.0959752321981426 %
3.2507739938080493 %
3.4055727554179565 %
3.5603715170278636 %
3.7151702786377707 %
3.869969040247678 %
4.024767801857585 %
4.179566563467493 %
4.3343653250774 %
4.489164086687307 %
4.643962848297214 %
4.798761609907121 %
4.953560371517028 %
5.108359133126935 %
5.2631578947368425 %
5.41795665634675 %
5.572755417956657 %
5.727554179566564 %
5.882352941176471 %
6.037151702786378 %
6.191950464396285 %
6.3467492260061915 %
6.501547987616099 %
6.656346749226006 %
6.811145510835913 %
6.96594427244582 %
7.120743034055727 %
7.275541795665634 %
7.430340557275541 %
7.585139318

62.693498452012385 %
62.84829721362229 %
63.0030959752322 %
63.1578947368421 %
63.31269349845201 %
63.46749226006192 %
63.62229102167183 %
63.77708978328173 %
63.93188854489164 %
64.08668730650155 %
64.24148606811146 %
64.39628482972137 %
64.55108359133126 %
64.70588235294117 %
64.86068111455108 %
65.015479876161 %
65.17027863777089 %
65.3250773993808 %
65.47987616099071 %
65.63467492260062 %
65.78947368421052 %
65.94427244582043 %
66.09907120743034 %
66.25386996904025 %
66.40866873065015 %
66.56346749226006 %
66.71826625386997 %
66.87306501547988 %
67.02786377708978 %
67.18266253869969 %
67.3374613003096 %
67.49226006191951 %
67.6470588235294 %
67.80185758513932 %
67.95665634674923 %
68.11145510835914 %
68.26625386996903 %
68.42105263157895 %
68.57585139318886 %
68.73065015479877 %
68.88544891640866 %
69.04024767801857 %
69.19504643962848 %
69.3498452012384 %
69.50464396284829 %
69.6594427244582 %
69.81424148606811 %
69.96904024767802 %
70.12383900928792 %
70.27863777089783 %
70.43343

In [8]:
label=np.array(label_arr)

In [11]:
# 保存到文件 避免下次重复计算
import scipy.io as io
data_mat_path = 'mats/data_old'
label_mat_path = 'labels/label_old'
io.savemat(data_mat_path, {'name': data})
io.savemat(label_mat_path, {'name': label})

# 从mat文件加载保存的图片和标签

In [25]:
# 加载data、label
import scipy.io as io
data_mat_path = 'mats/'
label_mat_path = 'labels/'

data_filename = 'data.mat'
label_filename = 'label.mat'
data = io.loadmat(data_mat_path+data_filename)["name"]
label = io.loadmat(label_mat_path+label_filename)["name"]

In [26]:
label = io.loadmat(label_mat_path+label_filename)["name"]

In [27]:
for i in range(64467):  # 去除因对齐添加的空格
    for j in range(16):
        label[i][j] = label[i][j].strip()

# 进行数据清洗

In [18]:
# 选取所有label作为使用，因为这样的估价会更加准确
# 首先对label进行编码，数字类型的和字符串类型的要使用不同的编码方式
# 参考 https://blog.csdn.net/luguanyou/article/details/80598122

In [28]:
# 先将label转换为DataFrame的格式,方便可视化观察
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from pandas import DataFrame

data_frame_label = DataFrame(label, index=None, columns=['Make', 'Module', 'Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
                                                         'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
                                                         'Drivetrain', 'PassengerCpacity', 'PassengerDoors', 'BodyStyle'])


In [29]:
data_frame_label

,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
0,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
1,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
2,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
3,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
4,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64462,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV
64463,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV
64464,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV
64465,Volvo,XC90,2020,50,19,250,20,4,79,69,194,18,AWD,7,4,SUV


In [30]:
row,col=data_frame_label.shape

In [31]:
data_frame_label.loc[256:256]

,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
256,Acura,MDX,2013,43,18,300,37,6,78,68,191,nan,AWD,7,4,SUV


In [32]:
# 这时候需要进行数据清洗，因为有空值的存在
# 先查看一下有多少记录存在空值

cols_list = ['Make', 'Module', 'Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
             'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
             'Drivetrain', 'PassengerCpacity', 'PassengerDoors', 'BodyStyle']
del_index = []
for cols in cols_list:
    for i in range(row):
        if data_frame_label[cols][i] == 'nan':
            del_index.append(i)
len(del_index)

24947

In [24]:
row

64467

In [33]:
# 可以看到有24947个记录存在nan
# 处理空值主要有两种方式：剔除、替换
# 在这里存在空值的记录占到了三分之一，用剔除法不太合适，所以决定进行均值填充
# 但是有四个字段是字符串类型，无法进行均值计算，我们选择保留它
import re

from fnmatch import fnmatchcase as match

num_cols_list = ['Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
                 'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
                 'PassengerCpacity', 'PassengerDoors']
# avg_year = avg_msrp = avg_fws = avg_horsr = avg_disp = avg_engine = avg_w = avg_h = avg_l = avg_gas = avg_passcap = avg_passdoor = 0
avg = [0 for i in range(len(num_cols_list))]
sums = [0 for i in range(len(num_cols_list))]
num = [0 for i in range(len(num_cols_list))]
for i in range(row):
    for j in range(len(num_cols_list)):
        x = data_frame_label[num_cols_list[j]][i]
#         if match(x,[0-9]\d*):
        if match(x,"[0-9]*"):
            num[j] += 1
            sums[j] += int(x)

In [34]:
num

[64467,
 63673,
 63732,
 63569,
 62257,
 62145,
 63609,
 63560,
 57776,
 57494,
 63740,
 63740]

In [35]:
sums

[129901542,
 3059783,
 1130543,
 17559500,
 1929296,
 339704,
 4660909,
 3928630,
 10685056,
 1214799,
 316266,
 223343]

In [36]:
for i in range(len(num_cols_list)):
    avg[i]=int(sums[i]/num[i])
avg

[2015, 48, 17, 276, 30, 5, 73, 61, 184, 21, 4, 3]

In [37]:
# 有了均值之后，对非法值进行修改
for i in range(row):
    for j in range(len(num_cols_list)):
        x = data_frame_label[num_cols_list[j]][i]
        if match(x, "[0-9]*"):
            data_frame_label[num_cols_list[j]][i]=int(x)
        else:
            data_frame_label[num_cols_list[j]][i]=avg[j]
    if i%1000!=0:
        pass
    else:
        print(100*i/row,"%")


0.0 %
1.5511812245024585 %
3.102362449004917 %
4.653543673507376 %
6.204724898009834 %
7.755906122512293 %
9.307087347014752 %
10.858268571517211 %
12.409449796019668 %
13.960631020522127 %
15.511812245024586 %
17.062993469527044 %
18.614174694029504 %
20.16535591853196 %
21.716537143034422 %
23.26771836753688 %
24.818899592039337 %
26.370080816541797 %
27.921262041044255 %
29.472443265546715 %
31.023624490049173 %
32.57480571455163 %
34.12598693905409 %
35.67716816355655 %
37.22834938805901 %
38.779530612561466 %
40.33071183706392 %
41.88189306156638 %
43.433074286068845 %
44.9842555105713 %
46.53543673507376 %
48.086617959576216 %
49.63779918407867 %
51.18898040858114 %
52.740161633083595 %
54.29134285758605 %
55.84252408208851 %
57.39370530659097 %
58.94488653109343 %
60.49606775559589 %
62.047248980098345 %
63.5984302046008 %
65.14961142910326 %
66.70079265360572 %
68.25197387810817 %
69.80315510261063 %
71.3543363271131 %
72.90551755161556 %
74.45669877611802 %
76.00788000062047 %

In [236]:

# 再查看一下有多少记录数值字段存在空值

del_index = []
for cols in num_cols_list:
    for i in range(row):
        if data_frame_label[cols][i] == 'nan':
            del_index.append(i)
len(del_index)

0

In [237]:
data_frame_label.head(100)

,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
0,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
1,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
2,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
3,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
4,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Acura,ILX,2017,27,17,200,24,4,70,55,181,25,FWD,5,4,4dr
96,Acura,ILX,2017,27,17,200,24,4,70,55,181,25,FWD,5,4,4dr
97,Acura,ILX,2017,27,17,200,24,4,70,55,181,25,FWD,5,4,4dr
98,Acura,ILX,2017,27,17,200,24,4,70,55,181,25,FWD,5,4,4dr


In [38]:
data_frame_label.shape

(64467, 16)

# 进行独热编码

In [31]:
# 然后对label中的不同标签分别使用sklearn的LabelBinarizer进行编码，替换原先label的标签
encoder = LabelBinarizer()

make_onehot = encoder.fit_transform(data_frame_label['Make'])
module_onehot = encoder.fit_transform(data_frame_label['Module'])
drivetrain_onehot = encoder.fit_transform(data_frame_label['Drivetrain'])
bodystyle_one_hot = encoder.fit_transform(data_frame_label['BodyStyle'])

In [32]:
# 现在已经将字符串类型的标签进行独热编码，放到四个变量当中。接下来，移除原dataframelabel中这四列
# 在移除之前，要先建立独热编码后的标签和之前的联系，用字典表示
dict_make = {}
dict_module = {}
dict_drivetrain = {}
dict_bodystyle = {}

# 要作为字典的key，应该将独热编码转换为string
n = len(make_onehot)  # 64467

str_make_one_hot = [''for i in range(n)]
str_module_one_hot = [''for i in range(n)]
str_drivetrain_onehot = [''for i in range(n)]
str_bodystyle_one_hot = [''for i in range(n)]
# tuple_make_one_hot = tuple(make_onehot)
# tuple_module_one_hot = tuple(module_onehot)
# tuple_drivetrain_onehot = tuple(drivetrain_onehot)
# tuple_bodystyle_one_hot = tuple(bodystyle_one_hot)


for i in range(n):
    for j in range(len(make_onehot[0])):
        str_make_one_hot[i] += str(make_onehot[i][j])
    for j in range(len(module_onehot[0])):
        str_module_one_hot[i] += str(module_onehot[i][j])
    for j in range(len(drivetrain_onehot[0])):
        str_drivetrain_onehot[i] += str(drivetrain_onehot[i][j])
    for j in range(len(bodystyle_one_hot[0])):
        str_bodystyle_one_hot[i] += str(bodystyle_one_hot[i][j])

    dict_make[str_make_one_hot[i]] = data_frame_label['Make'][i].strip()
    dict_module[str_module_one_hot[i]] = data_frame_label['Module'][i].strip()
    dict_drivetrain[str_drivetrain_onehot[i]
                    ] = data_frame_label['Drivetrain'][i].strip()
    dict_bodystyle[str_bodystyle_one_hot[i]
                   ] = data_frame_label['BodyStyle'][i].strip()

In [33]:
# 移除四列,保存在新变量data_frame_label_nums 中
data_frame_label_nums = data_frame_label.drop(
    ['Make', 'Module', 'Drivetrain', 'BodyStyle'], axis=1)

In [34]:
# 现在想办法将移除的四列重新以独热编码的格式插入到data_frame_label中去

# 查看data_frame_label_num的格式
data_frame_label_nums.head()

,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,PassengerCpacity,PassengerDoors
0,2013,28,16,110,15,4,70,55,179,39,5,4
1,2013,28,16,110,15,4,70,55,179,39,5,4
2,2013,28,16,110,15,4,70,55,179,39,5,4
3,2013,28,16,110,15,4,70,55,179,39,5,4
4,2013,28,16,110,15,4,70,55,179,39,5,4


In [35]:
print(type(data_frame_label_nums),type(make_onehot))

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [36]:
# 可以看到一个是DataFrame类型，另外四个是ndarray类型，所以我们首先将类型统一
data_ndarray_label_nums=data_frame_label_nums.values

In [37]:
data_ndarray_label_nums

array([[2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       ...,
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4]], dtype=object)

In [38]:
type(data_frame_label_nums['Year'][277])

int

In [178]:
# #去空格，转为数值型
# for i in range(64467):  # 去除因对齐添加的空格
#     for j in range(12):
#         data_ndarray_label_nums[i][j] = int(data_ndarray_label_nums[i][j].strip())

In [287]:
# sum=0
# num=0
# for i in range(64467):
#     if data_frame_label_nums['GasMileage'][i]!='nan':
#         sum+=int(data_frame_label_nums['GasMileage'][i])
#         num+=1
# avg=int(sum/num)
# avg

In [286]:
# # 由于存在nan，无法转换为int，所以对nan进行替换，替换为平均数

# for i in range(64467):  
#     for j in range(12):
#         if data_ndarray_label_nums[i][j] != 'nan':
#             data_ndarray_label_nums[i][j] = int(data_ndarray_label_nums[i][j])
#         else:
#             data_ndarray_label_nums[i][j] = avg

In [307]:
# 格式统一之后，为了进行拼接，需要对shape统一
data_ndarray_label_nums.shape

(64467, 12)

In [289]:
make_onehot.shape

(64467, 42)

In [304]:
module_onehot.shape

(64467, 323)

In [305]:
drivetrain_onehot.shape

(64467, 5)

In [306]:
bodystyle_one_hot.shape

(64467, 9)

In [332]:
# 一种实现思路是将data_ndarray_label_nums进行升维，如将3变为[3]，这样就可以和其他四个独热编码同构(需要把其它四个也升三维），可以进行拼接
# 另一种实现思路是直接将5个array进行拼接，表头的含义通过字典进行查找

# 由于第一种方案更接近人的思想，我们优先考虑这种

In [308]:
data_ndarray_label_nums

array([[2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       [2013, 28, 16, ..., 39, 5, 4],
       ...,
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4],
       [2020, 50, 19, ..., 18, 7, 4]], dtype=object)

In [317]:
data_ndarray_label_nums=data_ndarray_label_nums.reshape(64467,12,1)

In [321]:
data_ndarray_label_nums.shape

(64467, 12, 1)

In [324]:
make_onehot=make_onehot.reshape(64467,1,-1)

In [327]:
make_onehot[0][0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [328]:
module_onehot=module_onehot.reshape(64467,1,-1)
drivetrain_onehot=drivetrain_onehot.reshape(64467,1,-1)
bodystyle_one_hot=bodystyle_one_hot.reshape(64467,1,-1)

In [336]:
bodystyle_one_hot[0]

array([[0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [335]:
data_ndarray_label_nums[0]

array([[2013],
       [28],
       [16],
       [110],
       [15],
       [4],
       [70],
       [55],
       [179],
       [39],
       [5],
       [4]], dtype=object)

In [333]:
data_ndarray_label_one_hot = np.c_[data_ndarray_label_nums, make_onehot]

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [39]:
# 看来第一种方案行不通，shape不同无法进行拼接

# 只好采取第二种方案
# 首先把这些数据reshape到原来的结构
data_ndarray_label_nums=data_ndarray_label_nums.reshape(64467,-1)
make_onehot=make_onehot.reshape(64467,-1)
module_onehot=module_onehot.reshape(64467,-1)
drivetrain_onehot=drivetrain_onehot.reshape(64467,-1)
bodystyle_one_hot=bodystyle_one_hot.reshape(64467,-1)

In [40]:
data_ndarray_label_one_hot = np.c_[
    data_ndarray_label_nums, make_onehot, module_onehot, drivetrain_onehot, bodystyle_one_hot]

In [41]:
data_ndarray_label_one_hot.shape

(64467, 391)

In [42]:
data_ndarray_label_one_hot

array([[2013, 28, 16, ..., 0, 0, 0],
       [2013, 28, 16, ..., 0, 0, 0],
       [2013, 28, 16, ..., 0, 0, 0],
       ...,
       [2020, 50, 19, ..., 0, 0, 0],
       [2020, 50, 19, ..., 0, 0, 0],
       [2020, 50, 19, ..., 0, 0, 0]], dtype=object)

## 这里得到label

In [43]:
row, width_make_one_hot = make_onehot.shape
row, width_module_one_hot = module_onehot.shape
row, width_drivetrain_one_hot = drivetrain_onehot.shape
row, width_bodystyle_one_hot = bodystyle_one_hot.shape
print(width_make_one_hot, width_module_one_hot,
      width_drivetrain_one_hot, width_bodystyle_one_hot)


# 这里要记住data_ndarray_label_one_hot每一列的含义
# 0: 12  ['Year', 'MSRP', 'FrontWheelSize', 'Horsepower', 'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
#         'PassengerCpacity', 'PassengerDoors']

# 12:12+42 Make
# 12+42:12+42+323 Module
# 12+42+323: 12+42+323+5  DriveTrain
# 12+42+323+5:    BodyStyle

42 323 5 9


In [28]:
# imgs = os.listdir('cardata')
# img = Image.open('cardata/'+imgs[0])

In [301]:
# a=imgs[0].split('_')[0:16]
# print(a)
# make 制造商                   0     +
# module 款式                   1     +
# year 出场年份                 2  ?
# MSRP  价格/千元               3     +
# FrontWheelSize 前轮尺寸       4  -
# Horsepower 马力               5     +
# Displacement 油箱             6     +
# EngineType 引擎种类           7  -
# Width  宽度                   8     +
# Heigh  高度                   9     +
# Length 长度                   10    +
# GasMileage 油耗               11    +
# Drivetrain 传动系统           12  -
# PassengerCpacity 载客人数     13    +
# Passenger Doors  车门数量     14    +
# Body Style  车身风格          15  -

# 进行训练

In [44]:
# 到此为止，所有的图片保存在了data中，所有的标签保存到了data_ndarray_label_one_hot
# 标签名字过长，我们进行简化
import copy
label=copy.deepcopy(data_ndarray_label_one_hot)

In [45]:
label.shape

(64467, 391)

## 打乱顺序

In [46]:
#打乱顺序
#读取data矩阵的第一维数（图片的个数）
num_example=data.shape[0]
#产生一个num_example范围，步长为1的序列
arr=np.arange(num_example)
#调用函数，打乱顺序
np.random.shuffle(arr)
#按照打乱的顺序，重新排序
data=data[arr]
label=label[arr]


## 划分训练集、测试集

In [103]:
r_train = 0.8


s_train = np.int(num_example * r_train)


X_train = data[:s_train]
y_train = label[:s_train]

X_test = data[s_train:]
y_test = label[s_train:]



In [104]:
X_train.shape

(51573, 60, 80, 3)

In [105]:
num_labels=y_train.shape[1]
num_labels

391

## 构建网络

In [51]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
#导入必备包
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(tf.test.is_gpu_available)
print(tf.test.is_built_with_cuda)
print(tf.test.gpu_device_name)
print(tf.test.is_built_with_gpu_support)


2.0.0
<function is_gpu_available at 0x0000029CD83540D8>
<function is_built_with_cuda at 0x0000029CD835F9D8>
<function gpu_device_name at 0x0000029CD8352798>
<function is_built_with_gpu_support at 0x0000029CD835FAF8>


In [52]:
tf.test.is_gpu_available

<function tensorflow.python.framework.test_util.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)>

In [53]:
import tensorflow as tf
# 测试GPU是否可用
tf.compat.v1.disable_eager_execution()
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.compat.v1.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


### 构建CNN基本单元Conv+BN+Relu类模块

In [54]:
class ConvBNRelu(keras.Model):
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()

        self.model = keras.models.Sequential([
            keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding),  # 卷积
            keras.layers.BatchNormalization(),                                    #批量归一化
            keras.layers.ReLU()                                                   #激活函数
        ])
        
    def call(self, x, training=None):
        x = self.model(x, training=training)
        return x


### 构建Inception Block模块

In [55]:
class InceptionBlk(keras.Model):

    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()                    # 父类方法

        self.ch = ch                                            # 赋值
        self.strides = strides                                  # 赋值

        self.conv1 = ConvBNRelu(ch, strides=strides)             # 构造第1个CNN基本单元
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides) # 构造第2个CNN基本单元，卷积核大小初始为3*3
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)# 构造第3_1个CNN基本单元，卷积核大小初始为3*3
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)      # 构造第3_2个CNN基本单元，卷积核大小初始为3*3

        self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same')  # 最大池化层，same
        self.pool_conv = ConvBNRelu(ch, strides=strides)                    #构造CNN基本单元，卷积核大小初始为3*3

    def call(self, x, training=None):
        x1 = self.conv1(x, training=training)

        x2 = self.conv2(x, training=training)

        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)

        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        # concat along axis=channel
        x = tf.concat([x1, x2, x3_2, x4], axis=3)                         # 通道数扩充，上图理解。
        return x



### 构建Res Block 模块

In [56]:
# Res Block 模块。继承keras.Model或者keras.Layer都可以
class Inception(keras.Model):

    def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
        super(Inception, self).__init__(**kwargs)

        self.in_channels = init_ch                      # 初始输入通道
        self.out_channels = init_ch                     # 初始输出通道数
        self.num_layers = num_layers                    # 初始层数，就是多少个res block
        self.init_ch = init_ch                          # 初始通道

        self.conv1 = ConvBNRelu(init_ch)                  # 构造1个CNN基本单元

        self.blocks = keras.models.Sequential(
            name='dynamic-blocks')  # 创建一个Sequential容器对象

        # for循环，循环次数为num_layers传输的参数；
        for block_id in range(num_layers):

            # 2个Inception Block模块构建一个参考块，循环2次；
            for layer_id in range(2):

                if layer_id == 0:                          # 如果为第1次循环的话

                    # 创建1个Inception Block模块，并且步长为2；
                    block = InceptionBlk(self.out_channels, strides=2)

                else:                                        # 如果为第2次循环的话
                    # 创建1个Inception Block模块，并且步长为1；
                    block = InceptionBlk(self.out_channels, strides=1)

                self.blocks.add(block)                   # 把block放进容器对象blocks中

            # enlarger out_channels per block            # 通道数扩大为2倍。
            self.out_channels *= 2

        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, x, training=None):

        out = self.conv1(x, training=training)

        out = self.blocks(out, training=training)

        out = self.avg_pool(out)
        out = self.fc(out)

        return out

In [376]:
# def conv_module(x,k,kx,ky,stride,chandim,padding="same"):
#     #conv-bn-relu
#     x = Conv2D(k,(kx,ky),strides=stride,padding=padding)(x)
#     x = BatchNormalization(axis=chandim)(x)
#     x = Activation('relu')(x)
#     return x
# def inception_module(x,num1x1,num3x3,chandim):
#     conv_1x1 = conv_module(x,num1x1,1,1,(1,1),chandim)
#     conv_3_3 = conv_module(x,num3x3,3,3,(1,1),chandim)
#     x = concatenate([conv_1x1,conv_3_3],axis=chandim)
#     return x
# def downsample_module(x,k,chandim):
#     #conv downsample and pool downsample
#     conv_3x3 = conv_module(x,k,3,3,(2,2),chandim,padding='valid')
#     pool = MaxPooling2D((3,3),strides=(2,2))(x)
#     x = concatenate([conv_3x3,pool],axis=chandim)
#     return x



In [377]:
# #然后定义整个MiniGoogLeNet
# def minigooglenet_functional(width,height,depth,classes):
#     inputshape=(height,width,depth)
#     chandim=-1
#     #define inputs and firse conv
#     inputs = Input(shape=inputshape)
#     x = conv_module(inputs,96,3,3,(1,1),chandim)
#     #def two inception and followed by a downsample
#     x = inception_module(x,32,32,chandim)
#     x = inception_module(x,32,48,chandim)
#     x = downsample_module(x,80,chandim)
#     #def four inception and one downsample
#     x = inception_module(x,112,48,chandim)
#     x = inception_module(x,96,64,chandim)
#     x = inception_module(x,80,80,chandim)
#     x = inception_module(x,48,96,chandim)
#     x = downsample_module(x,96,chandim)
#     #def two inception followed by global pool and dropout
#     x = inception_module(x,176,160,chandim)
#     x = inception_module(x,176,160,chandim)
#     x = AveragePooling2D((7,7))(x)
#     x = Dropout(0.5)(x)
#     #softmax
#     x = Flatten()(x)
#     x = Dense(classes)(x)
#     x = Activation('softmax')(x)

#     #create the model
#     model = Model(inputs,x,name='MiniGoogLeNet')
#     return model


In [381]:
# model = minigooglenet_functional(width, height, channel, num_labels)

In [364]:

# import tensorflow as tf


# class Model(object):
#     def __init__(self, batch_size=128, learning_rate=1e-3, num_labels=391):
#         self._batch_size = batch_size
#         self._learning_rate = learning_rate
#         self._num_labels = num_labels

#     def inference(self, images, keep_prob):
#         #第一层卷积层（含激活层）
#         with tf.variable_scope('conv1') as scope:
#             kernel = self._create_weights([5, 5, 1, 32])# 5*5卷积核
#             conv = self._create_conv2d(images, kernel)
#             bias = self._create_bias([32])
#             preactivation = tf.nn.bias_add(conv, bias)
#             conv1 = tf.nn.relu(preactivation, name=scope.name)
#             self._activation_summary(conv1)

#         # pool 1
#         h_pool1 = self._create_max_pool_2x2(conv1)
#         #第二层卷积层（含激活层）
#         with tf.variable_scope('conv2') as scope:
#             kernel = self._create_weights([5, 5, 32, 64])
#             conv = self._create_conv2d(h_pool1, kernel)
#             bias = self._create_bias([64])
#             preactivation = tf.nn.bias_add(conv, bias)
#             conv2 = tf.nn.relu(preactivation, name=scope.name)
#             self._activation_summary(conv2)

#         # pool 2
#         h_pool2 = self._create_max_pool_2x2(conv2)

#         with tf.variable_scope('local1') as scope:
#             reshape = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
#             W_fc1 = self._create_weights([7 * 7 * 64, 1024])
#             b_fc1 = self._create_bias([1024])
#             local1 = tf.nn.relu(tf.matmul(reshape, W_fc1) + b_fc1, name=scope.name)
#             self._activation_summary(local1)

#         with tf.variable_scope('local2_linear') as scope:
#             W_fc2 = self._create_weights([1024, self._num_labels])
#             b_fc2 = self._create_bias([self._num_labels])
#             local1_drop = tf.nn.dropout(local1, keep_prob)
#             local2 = tf.nn.bias_add(tf.matmul(local1_drop, W_fc2), b_fc2, name=scope.name)
#             self._activation_summary(local2)
#         return local2

#     def train(self, loss, global_step):
#         tf.summary.scalar('learning_rate', self._learning_rate)
#         train_op = tf.train.AdamOptimizer(self._learning_rate).minimize(loss, global_step=global_step)
#         return train_op

#     def loss(self, logits, labels):
#         with tf.variable_scope('loss') as scope:
#             cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
#             cost = tf.reduce_mean(cross_entropy, name=scope.name)
#             tf.summary.scalar('cost', cost)

#         return cost

#     def accuracy(self, logits, labels):
#         with tf.variable_scope('accuracy') as scope:
#             accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1)), 
#             dtype=tf.float32), name=scope.name)
#             tf.summary.scalar('accuracy', accuracy)
#         return accuracy

#     def _create_conv2d(self, x, W):
#         return tf.nn.conv2d(input=x,
#                             filter=W,
#                             strides=[1, 1, 1, 1],
#                             padding='SAME')

#     def _create_max_pool_2x2(self, input):
#         return tf.nn.max_pool(value=input,
#                               ksize=[1, 2, 2, 1],
#                               strides=[1, 2, 2, 1],
#                               padding='SAME')

#     def _create_weights(self, shape):
#         return tf.Variable(tf.truncated_normal(shape=shape, stddev=0.1, dtype=tf.float32))#标准差为0.1的正态分布函数

#     def _create_bias(self, shape):
#         return tf.Variable(tf.constant(1., shape=shape, dtype=tf.float32))

#     def _activation_summary(self, x):
#         tensor_name = x.op.name
#         tf.summary.histogram(tensor_name + '/activations', x)
#         tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

In [365]:
# Inference 这个函数创建对输入表示创建一个预测。它对每个输入返回一个 1×10的张量。张量中的值会传递给损失函数，计算这个预测与基准相差多少。

# 根据参数batch_size，每次需要处理128个图像。这个技术称为mini-batch。
# 在更小的数据集上batch上处理输入数据，而不是整个数据集，这样就可以有效使用内存。
# 因为是在每个小型数据集上更新权重，而不是处理所有样本，模型也能够更快收敛。

# Loss 可以用softmax cross entropy 函数执行N路分类。Softmax可以归一化（将tensor加成1个数值）从inference函数产生的输入数据。

# 张量归一化后，Cross entropy 计算编码标注。cross entropy计算预测值与基准值之间的差异。每次迭代，优化器都最小化cross entropy的值。


## 开始训练

In [57]:
import tensorflow as tf
from tensorflow.python.keras.api._v2.keras import layers, optimizers, datasets, Sequential
import tensorflow.keras as keras
import numpy as np

import os

In [58]:
# 调用Inception
model = Inception(2, num_labels)   # 第一参数为残差块数，第二个参数为类别数；
# derive input shape for every layers.
# print(width,height,channel)
model.build(input_shape=(None, width, height, channel))
model.summary()


60 80 3
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "inception"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu (ConvBNRelu)    multiple                  512       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  292704    
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  50439     
Total params: 343,655
Trainable params: 342,663
Non-trainable params: 992
_________________________________________________________________


In [90]:
# X=tf.constant(X_train[:10000],dtype=tf.int32,name="X")

In [91]:
# y=tf.constant(y_train[:10000],dtype=tf.int32,name="y")

In [92]:
# XT=tf.transpose(X)

In [87]:
batch_size, height, width, channel = X_train.shape

In [98]:
batch_size=8000

In [94]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [95]:
check_path = './ckpt/cp-{epoch:04d}.ckpt'

In [97]:
save_model_cb = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True, verbose=1, save_freq=5)

In [100]:
model.fit(X_train, y_train, epochs=5, callbacks=[save_model_cb])
test_loss, test_acc = model.evaluate(X_test, y_test)
print("准确率: %.4f，共测试了%d张图片 " % (test_acc, len(y_test)))

Train on 38680 samples
Epoch 1/5


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Can not squeeze dim[1], expected a dimension of 1, got 391
	 [[{{node metrics/accuracy/Squeeze}}]]
	 [[Func/training/Adam/gradients/gradients/inception/conv_bn_relu/sequential/batch_normalization/cond_grad/If/then/_1776/input/_4540/_1355]]
  (1) Invalid argument: Can not squeeze dim[1], expected a dimension of 1, got 391
	 [[{{node metrics/accuracy/Squeeze}}]]
0 successful operations.
0 derived errors ignored.

In [163]:
# 由于tensorflow仅支持一维向量作为label，所以要对独热编码进行处理
# 一种简单的思路是为16种不同的角度训练16个分类器（独热编码可以用argmax转换为索引），但是这看起来有点蠢。
# 所以我决定找一个方式实现多标签分类

#  这个问题本身既包含分类问题，又包含回归问题，不可笼统处理。
#查阅资料后得出最终方案：
# 训练一个分类器（将汽车生产厂家和型号作为类别，即Make和Model两个组合在一起作为一个标签），这作为一个多分类问题进行处理，
# 得到的结果为onehot编码。查阅字典即可转换为字符串。

# 再训练一个回归器，将分类器的输出和其他13个维度组合在一起作为回归器的输入，价格作为输出。
#


#需要说明的是我们放弃了4.1，5.1，5.2，5.3，5.4节的一切操作，我们从第6节开始接着第三节重新进行训练



# 重新开始训练

In [39]:
# 训练之前，先做一下标签和数据整理
y_train.shape


NameError: name 'y_train' is not defined

## 标签整理

In [42]:
print(len(data_frame_label))
data_frame_label.head()# 这里的dataframe——label是第三节末生成的数据

64467


,Make,Module,Year,MSRP,FrontWheelSize,Horsepower,Displacement,EngineType,Width,Heigh,Length,GasMileage,Drivetrain,PassengerCpacity,PassengerDoors,BodyStyle
0,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
1,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
2,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
3,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr
4,Acura,ILX,2013,28,16,110,15,4,70,55,179,39,FWD,5,4,4dr


In [43]:
row

64467

In [44]:
y_label=[''for i in range(row)]

In [45]:
data_frame_make = data_frame_label['Make']

In [46]:
data_frame_module = data_frame_label['Module']

In [47]:
data_frame_make[10000]

'Buick'

In [ ]:
for i in range(row):
    y_label[i] = data_frame_make[i]+'_'+data_frame_module[i]

In [ ]:
y_label[9999]

## 数据整理

In [48]:
data.shape

(64467, 60, 80, 3)

In [ ]:
X = data

## 独热编码

In [ ]:
encoder = LabelBinarizer()
y_onehot = encoder.fit_transform(y_label)

In [ ]:
y_onehot.shape

In [ ]:
y_onehot[0]

In [ ]:
y=y_onehot

## 划分训练集、测试集

In [ ]:
# 我们不在使用5.1和5.2生成的训练集和测试集，而是重新进行训练集测试集的划分

In [ ]:
import numpy as np
#打乱顺序
#读取X矩阵的第一维数（图片的个数）
num_example=X.shape[0]
#产生一个num_example范围，步长为1的序列
arr=np.arange(num_example)
#调用函数，打乱顺序
np.random.shuffle(arr)
#按照打乱的顺序，重新排序
X=X[arr]
y=y[arr]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
r_train = 0.8


s_train = np.int(num_example * r_train)


X_train = X[:s_train]
y_train = y[:s_train]

X_test = X[s_train:]
y_test = y[s_train:]


In [ ]:
#归一化
X_train=X_train/255
X_test=X_test/255

In [ ]:
X[8][0]

In [ ]:
X_train[8888][0]

In [ ]:
X_test.shape

In [ ]:
y_train.shape

## 构造网络

In [25]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
#导入必备包
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt


In [26]:
# 卷积层


def conv_module(x, filters, input_shape, strides, kernel_size, padding="same", activation='relu'):
    # conv-bn-relu
    x = Conv2D(filters=filters,
               input_shape=input_shape,
               kernel_size=kernel_size,
               strides=strides,
               activation=activation,
               padding=padding
               )(x)
    return x

# 初始化层


def inception_module(x, num1x1, num3x3, input_shape, chandim=-1):
    conv_1x1 = conv_module(x=x,
                           filters=num1x1,
                           kernel_size=(1, 1),
                           strides=(1, 1),
                           input_shape=input_shape
                           )
    conv_3_3 = conv_module(x=x,
                           filters=num3x3,
                           kernel_size=(3, 3),
                           strides=(1, 1),
                           input_shape=input_shape
                           )
    x = concatenate([conv_1x1, conv_3_3], axis=chandim)
    return x

# 最大值池化层


def max_pool_module(x, pool_size, strides, padding='same'):
    x = MaxPooling2D(pool_size=pool_size,
                     strides=strides,
                     padding=padding,
                     )(x)
    return x

# 平均值池化层


def avg_pool_module(x, pool_size, strides, padding='valid'):
    x = AveragePooling2D(pool_size=pool_size,
                         strides=strides,
                         padding=padding,
                         )(x)
    return x

# def downsample_module(x, k, chandim=-1):
#     # conv downsample and pool downsample
#     conv_3x3 = conv_module(x, k, 3, 3, (2, 2), chandim, padding='valid')
#     pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
#     x = concatenate([conv_3x3, pool], axis=chandim)
#     return x

In [27]:
# 然后定义整个神经网络


def self_def_mam_model(batch_size, width, height, channel, classes):
    input_shape = (height, width, channel)
    chandim = -1

   # 定义神经网络各层
    #inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape, batch_size=batch_size)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 局部响应归一化
    x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 局部响应归一化
    x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 最大池化层192，(3,3)，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=64, num3x3=64, chandim=-1,input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=128, num3x3=128, chandim=-1,input_shape=input_shape)
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 初始化层
    x = inception_module(x=x, num1x1=192, num3x3=192, chandim=-1,input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=256, num3x3=256, chandim=-1,input_shape=input_shape)
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=320, num3x3=320, chandim=-1,input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=386, num3x3=386, chandim=-1,input_shape=input_shape)
    # 平均池化7*7,1V
    x = avg_pool_module(x=x, pool_size=(7, 7), strides=(1, 1), padding='valid')
    # dropout 40%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='MaM_Model')
    return model


#     x = conv_module(inputs, 96, 3, 3, (1, 1), chandim)
#     # def two inception and followed by a downsample
#     x = inception_module(x, 32, 32, chandim)
#     x = inception_module(x, 32, 48, chandim)
#     x = downsample_module(x, 80, chandim)
#     # def four inception and one downsample
#     x = inception_module(x, 112, 48, chandim)
#     x = inception_module(x, 96, 64, chandim)
#     x = inception_module(x, 80, 80, chandim)
#     x = inception_module(x, 48, 96, chandim)
#     x = downsample_module(x, 96, chandim)
#     # def two inception followed by global pool and dropout
#     x = inception_module(x, 176, 160, chandim)
#     x = inception_module(x, 176, 160, chandim)
#     x = AveragePooling2D((7, 7))(x)
#     x = Dropout(0.5)(x)
#     # softmax
#     x = Flatten()(x)
#     x = Dense(classes)(x)
#     x = Activation('softmax')(x)

#     # create the model
#     model = Model(inputs, x, name='MiniGoogLeNet')
#     return model

In [ ]:
batch_size=32
classes=y.shape[1]
row,height,width,channel=X_train.shape

In [ ]:
channel

In [ ]:
mam_model=self_def_mam_model(batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)

In [ ]:
# # Sequential 用于建立序列模型
# # Flatten 层用于展开张量，input_shape 定义输入形状为 28x28 的图像，展开后为 28*28 的张量。
# # Dense 层为全连接层，输出有 128 个神经元，激活函数使用 relu。
# # Dropout 层使用 0.2 的失活率。
# # 再接一个全连接层，激活函数使用 softmax，得到对各个类别预测的概率。
# model_mam = keras.Sequential()
# # 卷积64，7*7+2S
# model_mam.add(layers.Conv2D(filters=64,
#                             input_shape=(128, 60, 80, 3),
#                             kernel_size=(7, 7),
#                             strides=2,
#                             activation='Relu',
#                             padding='same',
#                            )
#              )
# # 最大池化 64，3*3+2S
# model_mam.add(layers.MaxPool2D(pool_size=(3, 3),
#                                strides=2,
#                                padding='same',
#                                data_format='tf.float32',
#                               )
#              )
# # 本地相应归一化LRN
# model_mam.add(tf.nn.lrn())

# model_mam.add(layers.Conv2D(())
# model_mam.add(layers.Flatten(input_shape=(28, 28)))
# model_mam.add(layers.Dense(128, activation="relu"))
# model_mam.add(layers.Dropout(0.2))
# model_mam.add(layers.Dense(10, activation="softmax"))

## 编译模型

In [172]:
# num_class=len(y[0])

In [173]:
# num_class#分类数

In [174]:
# rows,  height,width, channel = X.shape

In [177]:
# print("rows=",rows,"height=",height,"width=",width,"channel=",channel)

In [176]:
# import tensorflow as tf
# from tensorflow.python.keras.api._v2.keras import layers, optimizers, datasets, Sequential
# import tensorflow.keras as keras
# import numpy as np

# import os
# # 
# # 调用Inception
# model = Inception(2, num_class)   # 第一参数为残差块数，第二个参数为类别数；
# # derive input shape for every layers.
# # print(width,height,channel)
# model.build(input_shape=(None, width, height, channel))
# model.summary()

In [41]:
# 优化器选择 rmsprop 优化器。
# 损失函数使用  categorical_crossentropy，
# 还有一个损失函数是sparse_categorical_crossentropy，两者的区别在于输入的真实标签的形式，
# sparse_categorical 输入的是整形的标签，例如 [1, 2, 3, 4]，categorical 输入的是 one-hot 编码的标签。
mam_model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

NameError: name 'mam_model' is not defined

## 训练模型

In [95]:
check_path = './ckpt/cp-{epoch:04d}.ckpt'

In [96]:
save_model_cb = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True, verbose=1, save_freq=5)

In [40]:
# fit 用于训练模型，对训练数据遍历一次为一个 epoch，这里遍历 5 次。
# evaluate 用于评估模型，返回的数值分别是损失和指标。
mam_model.fit(X_train, y_train, epochs=5)
# mam_model.evaluate(X_test, y_test)

NameError: name 'mam_model' is not defined

In [ ]:
# 收敛速度过慢，我们对其进行优化

## 优化


In [ ]:
#带你跨过神经网络训练常见的37个坑
# https://www.cnblogs.com/ocean1100/articles/9034008.html

### 调整batch_size

In [113]:
import gc
gc.collect()

209

In [118]:
batch_size=32

In [119]:
mam_model=self_def_mam_model(batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)

In [120]:
mam_model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
mam_model.fit(X_train, y_train, epochs=5)
# mam_model.evaluate(X_test, y_test)

In [228]:
mam_model.fit(X_train[:128], y_train[:128], epochs=50)

Train on 128 samples
Epoch 1/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.1138 - accuracy: 0.9688
Epoch 2/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.0845 - accuracy: 0.9766
Epoch 3/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.1074 - accuracy: 0.9688
Epoch 4/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.1301 - accuracy: 0.9766
Epoch 5/50
128/128 [==============================] - 1s 7ms/sample - loss: 2.3724 - accuracy: 0.5781
Epoch 6/50
128/128 [==============================] - 1s 7ms/sample - loss: 1.7512 - accuracy: 0.7266
Epoch 7/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.2078 - accuracy: 0.9453
Epoch 8/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.0890 - accuracy: 0.9922
Epoch 9/50
128/128 [==============================] - 1s 7ms/sample - loss: 0.0968 - accuracy: 0.9922
Epoch 10/50
128/128 [==============================] - 1s 7ms

In [209]:
mam_model.evaluate(X_train,y_train)

51573/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[14.38659593326683, 0.06012836]

In [ ]:
### 可见虽然调低了batch_size收敛速度仍是不乐观，模型出现严重欠拟合。

### 减少模型中的正则化

In [126]:
gc.collect()

4591

In [122]:
def self_def_mam_model_cut(batch_size, width, height, channel, classes):
    input_shape = (height, width, channel)
    chandim = -1

   # 定义神经网络各层
    #inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape, batch_size=batch_size)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 局部响应归一化
    x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 局部响应归一化
    x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 最大池化层192，(3,3)，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=64, num3x3=64,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=128, num3x3=128,
                         chandim=-1, input_shape=input_shape)
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 初始化层
    x = inception_module(x=x, num1x1=192, num3x3=192,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=256, num3x3=256,
                         chandim=-1, input_shape=input_shape)
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=320, num3x3=320,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=386, num3x3=386,
                         chandim=-1, input_shape=input_shape)
    # 平均池化7*7,1V
    x = avg_pool_module(x=x, pool_size=(7, 7), strides=(1, 1), padding='valid')
    # dropout 40%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='MaM_Model')
    return model


#     x = conv_module(inputs, 96, 3, 3, (1, 1), chandim)
#     # def two inception and followed by a downsample
#     x = inception_module(x, 32, 32, chandim)
#     x = inception_module(x, 32, 48, chandim)
#     x = downsample_module(x, 80, chandim)
#     # def four inception and one downsample
#     x = inception_module(x, 112, 48, chandim)
#     x = inception_module(x, 96, 64, chandim)
#     x = inception_module(x, 80, 80, chandim)
#     x = inception_module(x, 48, 96, chandim)
#     x = downsample_module(x, 96, chandim)
#     # def two inception followed by global pool and dropout
#     x = inception_module(x, 176, 160, chandim)
#     x = inception_module(x, 176, 160, chandim)
#     x = AveragePooling2D((7, 7))(x)
#     x = Dropout(0.5)(x)
#     # softmax
#     x = Flatten()(x)
#     x = Dense(classes)(x)
#     x = Activation('softmax')(x)

#     # create the model
#     model = Model(inputs, x, name='MiniGoogLeNet')
#     return model

In [123]:
mam_model_cut = self_def_mam_model(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)

In [124]:
mam_model_cut.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [125]:
mam_model_cut.fit(X_train, y_train, epochs=5)
mam_model_cut.evaluate(X_test, y_test)

Train on 51573 samples
Epoch 1/5
51573/51573 [==============================] - 347s 7ms/sample - loss: 5.7143 - accuracy: 0.0073
Epoch 2/5
51573/51573 [==============================] - 345s 7ms/sample - loss: 5.5592 - accuracy: 0.0084
Epoch 3/5
51573/51573 [==============================] - 344s 7ms/sample - loss: 5.5570 - accuracy: 0.0086
Epoch 4/5
51573/51573 [==============================] - 344s 7ms/sample - loss: 5.5560 - accuracy: 0.0086
Epoch 5/5
51573/51573 [==============================] - 344s 7ms/sample - loss: 5.5549 - accuracy: 0.0086
12894/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================

[5.554053399624594, 0.007367768]

In [207]:
mam_model_cut.fit(X_train[:128], y_train[:128], epochs=100)

Train on 128 samples
Epoch 1/100
128/128 [==============================] - 3s 22ms/sample - loss: 5.6113 - accuracy: 0.0000e+00
Epoch 2/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.6086 - accuracy: 0.0000e+00
Epoch 3/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.6060 - accuracy: 0.0000e+00
Epoch 4/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.6033 - accuracy: 0.0000e+00
Epoch 5/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.6006 - accuracy: 0.0000e+00
Epoch 6/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.5977 - accuracy: 0.0000e+00
Epoch 7/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.5947 - accuracy: 0.0000e+00
Epoch 8/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.5917 - accuracy: 0.0000e+00
Epoch 9/100
128/128 [==============================] - 1s 7ms/sample - loss: 5.5886 - accuracy: 0.0000e+00
Epoch 10/100
12

### 增加模型深度

In [127]:
# 模型还是不收敛，增加模型复杂度试试看

In [132]:
# 然后定义整个神经网络


def self_def_mam_model(batch_size, width, height, channel, classes):
    input_shape = (height, width, channel)
    chandim = -1

   # 定义神经网络各层
    #inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape, batch_size=batch_size)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 局部响应归一化
    #x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 局部响应归一化
    #x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 最大池化层192，(3,3)，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=64, num3x3=64,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=128, num3x3=128,
                         chandim=-1, input_shape=input_shape)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 初始化层
    x = inception_module(x=x, num1x1=192, num3x3=192,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=256, num3x3=256,
                         chandim=-1, input_shape=input_shape)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
     # 初始化层
    x = inception_module(x=x, num1x1=64, num3x3=64,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=128, num3x3=128,
                         chandim=-1, input_shape=input_shape)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)

    # 初始化层
    x = inception_module(x=x, num1x1=320, num3x3=320,
                         chandim=-1, input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=386, num3x3=386,
                         chandim=-1, input_shape=input_shape)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 平均池化7*7,1V
    x = avg_pool_module(x=x, pool_size=(7, 7), strides=(1, 1), padding='valid')
    # dropout 40%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='MaM_Model')
    return model

In [133]:
mam_model_dp = self_def_mam_model(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)
mam_model_dp.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [154]:
batch_size=16
gc.collect()

18199

In [206]:
mam_model_dp.fit(X_train[:128], y_train[:128], epochs=100)

Train on 128 samples
Epoch 1/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.3655 - accuracy: 0.4453
Epoch 2/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.3559 - accuracy: 0.3984
Epoch 3/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.2729 - accuracy: 0.4375
Epoch 4/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.2779 - accuracy: 0.4062
Epoch 5/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.2896 - accuracy: 0.4141
Epoch 6/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.2375 - accuracy: 0.4688
Epoch 7/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.1790 - accuracy: 0.4531
Epoch 8/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.2060 - accuracy: 0.4609
Epoch 9/100
128/128 [==============================] - 0s 2ms/sample - loss: 3.1207 - accuracy: 0.4609
Epoch 10/100
128/128 [==============================

In [156]:
# mam_model_dp.fit(X_train, y_train, epochs=10)

In [157]:
mam_model_dp.evaluate(X_test, y_test)

12894/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[4.951440338905752, 0.110749185]

### 增加模型复杂度

In [194]:
# 卷积层


def conv_module(x, filters,  strides, kernel_size, padding="same", activation='relu'):
    # conv-bn-relu
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               activation=activation,
               padding=padding
               )(x)
    return x

# 初始化层


def inception_module(x, num1x1_1, num1x1_2, num1x1_3, num1x1_4, num3x3, num_5x5, chandim=-1):
    #
    conv_1x1_1 = conv_module(x=x,
                             filters=num1x1_1,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    conv_1x1_2 = conv_module(x=x,
                             filters=num1x1_2,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    #
    conv_3_3 = conv_module(x=conv_1x1_2,
                           filters=num3x3,
                           kernel_size=(3, 3),
                           strides=(1, 1),
                           #                            input_shape=input_shape
                           )
    conv_1x1_3 = conv_module(x=x,
                             filters=num1x1_3,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    #
    conv_5_5 = conv_module(x=conv_1x1_3,
                           filters=num_5x5,
                           kernel_size=(5, 5),
                           strides=(1, 1),
                           #                            input_shape=input_shape
                           )
    max_pool = max_pool_module(x=x, pool_size=(
        3, 3), strides=(1, 1), padding='same')
    #
    conv_1x1_4 = conv_module(x=max_pool,
                             filters=num1x1_4,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )

    x = concatenate([conv_1x1_1, conv_3_3, conv_5_5, conv_1x1_4], axis=chandim)
    return x

# 最大值池化层


def max_pool_module(x, pool_size, strides,  padding='same'):
    x = MaxPooling2D(pool_size=pool_size,
                     strides=strides,
                     padding=padding,
                     # filters=filters
                     )(x)
    return x

# 平均值池化层


def avg_pool_module(x, pool_size, strides, padding='valid'):
    x = AveragePooling2D(pool_size=pool_size,
                         strides=strides,
                         padding=padding,
                         # filters=filters
                         )(x)
    return x


def self_def_mam_model_2(batch_size, width, height, channel, classes):

    input_shape = (height, width, channel)
    chandim = -1
   # 定义神经网络各层
    # inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape, batch_size=batch_size)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))
    # 局部响应归一化
#     x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192,
                    strides=(1, 1), kernel_size=(3, 3))
    # 局部响应归一化
#     x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 最大池化层192，(3,3)，2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1_1=64, num1x1_2=96, num1x1_3=16, num1x1_4=32,
                         num3x3=128, num_5x5=32)
    # 初始化层
    x = inception_module(x=x, num1x1_1=128, num1x1_2=128, num1x1_3=32, num1x1_4=64,
                         num3x3=192, num_5x5=96, chandim=-1)
    # 最大池化层,480,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1_1=192, num1x1_2=96, num1x1_3=16, num1x1_4=64,
                         num3x3=208, num_5x5=48, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=160, num1x1_2=112, num1x1_3=24, num1x1_4=64,
                         num3x3=224, num_5x5=64, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=128, num1x1_2=128, num1x1_3=24, num1x1_4=64,
                         num3x3=256, num_5x5=64, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=112, num1x1_2=144, num1x1_3=32, num1x1_4=64, num3x3=288, num_5x5=64,
                         chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=256, num1x1_2=160, num1x1_3=32, num1x1_4=128,
                         num3x3=320, num_5x5=128, chandim=-1)
    # 最大池化层,832,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1_1=64, num1x1_2=96, num1x1_3=16, num1x1_4=32,
                         num3x3=128, num_5x5=32, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=64, num1x1_2=96, num1x1_3=16, num1x1_4=32,
                         num3x3=128, num_5x5=32, chandim=-1)
    # 平均池化1024,7*7,1V
    x = avg_pool_module(x=x, pool_size=(7, 7), strides=(1, 1),
                         padding='valid')
    # dropout 40%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='MaM_Model')
    return model

In [197]:
mam_model_2 = self_def_mam_model_2(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)
mam_model_2.compile(optimizer="rmsprop",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])
batch_size = 16
gc.collect()
mam_model_2.fit(X_train, y_train, epochs=1)
# mam_model_2.evaluate(X_test, y_test)

Train on 51573 samples
51573/51573 [==============================] - 249s 5ms/sample - loss: 5.6006 - accuracy: 0.0076


In [202]:
mam_model_2.fit(X_train[:128], y_train[:128], epochs=50)

Train on 128 samples
Epoch 1/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7652 - accuracy: 0.0156
Epoch 2/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7758 - accuracy: 0.0078
Epoch 3/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7756 - accuracy: 0.0391
Epoch 4/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7411 - accuracy: 0.0234
Epoch 5/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7371 - accuracy: 0.0312
Epoch 6/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7166 - accuracy: 0.0156
Epoch 7/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7406 - accuracy: 0.0078
Epoch 8/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7273 - accuracy: 0.0312
Epoch 9/50
128/128 [==============================] - 1s 5ms/sample - loss: 4.7230 - accuracy: 0.0312
Epoch 10/50
128/128 [==============================] - 1s 5ms

In [203]:
#增大复杂度后，不能收敛，模型精度反而下降。所以，减小复杂度试试看

### 减小模型复杂度

In [198]:
# 卷积层


def conv_module(x, filters,  strides, kernel_size, padding="same", activation='relu'):
    # conv-bn-relu
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               activation=activation,
               padding=padding
               )(x)
    return x

# 初始化层


def inception_module(x, num1x1_1, num1x1_2, num1x1_3, num1x1_4, num3x3, num_5x5, chandim=-1):
    #
    conv_1x1_1 = conv_module(x=x,
                             filters=num1x1_1,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    conv_1x1_2 = conv_module(x=x,
                             filters=num1x1_2,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    #
    conv_3_3 = conv_module(x=conv_1x1_2,
                           filters=num3x3,
                           kernel_size=(3, 3),
                           strides=(1, 1),
                           #                            input_shape=input_shape
                           )
    conv_1x1_3 = conv_module(x=x,
                             filters=num1x1_3,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    #
    conv_5_5 = conv_module(x=conv_1x1_3,
                           filters=num_5x5,
                           kernel_size=(5, 5),
                           strides=(1, 1),
                           #                            input_shape=input_shape
                           )
    max_pool = max_pool_module(x=x, pool_size=(
        3, 3), strides=(1, 1), padding='same')
    #
    conv_1x1_4 = conv_module(x=max_pool,
                             filters=num1x1_4,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )

    x = concatenate([conv_1x1_1, conv_3_3, conv_5_5, conv_1x1_4], axis=chandim)
    return x

# 最大值池化层


def max_pool_module(x, pool_size, strides,  padding='same'):
    x = MaxPooling2D(pool_size=pool_size,
                     strides=strides,
                     padding=padding,
                     # filters=filters
                     )(x)
    return x

# 平均值池化层


def avg_pool_module(x, pool_size, strides, padding='valid'):
    x = AveragePooling2D(pool_size=pool_size,
                         strides=strides,
                         padding=padding,
                         # filters=filters
                         )(x)
    return x


def self_def_mam_model_3(batch_size, width, height, channel, classes):

    input_shape = (height, width, channel)
    chandim = -1
   # 定义神经网络各层
    # inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape, batch_size=batch_size)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64,
                    strides=(2, 2), kernel_size=(7, 7))
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))

    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64,
                    strides=(2, 2), kernel_size=(7, 7))
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64,
                    strides=(2, 2), kernel_size=(7, 7))

    # 最大池化层,480,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))

    # dropout 50%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='MaM_Model')
    return model

In [199]:
mam_model_3 = self_def_mam_model_2(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)
mam_model_3.compile(optimizer="rmsprop",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])
batch_size = 16
gc.collect()
mam_model_3.fit(X_train, y_train, epochs=1)
# mam_model_2.evaluate(X_test, y_test)

Train on 51573 samples
51573/51573 [==============================] - 265s 5ms/sample - loss: 5.5988 - accuracy: 0.0076


In [204]:
mam_model_3.fit(X_train[:128], y_train[:128], epochs=30)

Train on 128 samples
Epoch 1/30
128/128 [==============================] - 5s 39ms/sample - loss: 5.6146 - accuracy: 0.0000e+00
Epoch 2/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.6053 - accuracy: 0.0000e+00
Epoch 3/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.6000 - accuracy: 0.0000e+00
Epoch 4/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.5950 - accuracy: 0.0000e+00
Epoch 5/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.5840 - accuracy: 0.0000e+00
Epoch 6/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.5756 - accuracy: 0.0000e+00
Epoch 7/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.5632 - accuracy: 0.0000e+00
Epoch 8/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.5570 - accuracy: 0.0000e+00
Epoch 9/30
128/128 [==============================] - 1s 5ms/sample - loss: 5.5460 - accuracy: 0.0000e+00
Epoch 10/30
128/128 [===

In [210]:
#可见减小复杂度和增大复杂度对准确率没什么大的影响
#以上五个模型中，仅用128个训练数据，mam_model收敛效果最好，

In [211]:
# 对model2更换优化器为Adagrad

### 更换优化器

In [217]:
## 卷积层

def conv_module(x, filters, input_shape, strides, kernel_size, padding="same", activation='relu'):
    # conv-bn-relu
    x = Conv2D(filters=filters,
               input_shape=input_shape,
               kernel_size=kernel_size,
               strides=strides,
               activation=activation,
               padding=padding
               )(x)
    return x

# 初始化层


def inception_module(x, num1x1, num3x3, input_shape, chandim=-1):
    conv_1x1 = conv_module(x=x,
                           filters=num1x1,
                           kernel_size=(1, 1),
                           strides=(1, 1),
                           input_shape=input_shape
                           )
    conv_3_3 = conv_module(x=x,
                           filters=num3x3,
                           kernel_size=(3, 3),
                           strides=(1, 1),
                           input_shape=input_shape
                           )
    x = concatenate([conv_1x1, conv_3_3], axis=chandim)
    return x

# 最大值池化层


def max_pool_module(x, pool_size, strides, padding='same'):
    x = MaxPooling2D(pool_size=pool_size,
                     strides=strides,
                     padding=padding,
                     )(x)
    return x

# 平均值池化层


def avg_pool_module(x, pool_size, strides, padding='valid'):
    x = AveragePooling2D(pool_size=pool_size,
                         strides=strides,
                         padding=padding,
                         )(x)
    return x

# def downsample_module(x, k, chandim=-1):
#     # conv downsample and pool downsample
#     conv_3x3 = conv_module(x, k, 3, 3, (2, 2), chandim, padding='valid')
#     pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
#     x = concatenate([conv_3x3, pool], axis=chandim)
#     return x

# 然后定义整个神经网络


def self_def_mam_model_4(batch_size, width, height, channel, classes):
    input_shape = (height, width, channel)
    chandim = -1

   # 定义神经网络各层
    #inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape, batch_size=batch_size)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 局部响应归一化
    x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192, input_shape=input_shape,
                    strides=(1, 1), kernel_size=(3, 3))
    # 局部响应归一化
    x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 最大池化层192，(3,3)，2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=64, num3x3=64, chandim=-1,input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=128, num3x3=128, chandim=-1,input_shape=input_shape)
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2),)
    # 初始化层
    x = inception_module(x=x, num1x1=192, num3x3=192, chandim=-1,input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=256, num3x3=256, chandim=-1,input_shape=input_shape)
    # 最大池化层,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1=320, num3x3=320, chandim=-1,input_shape=input_shape)
    # 初始化层
    x = inception_module(x=x, num1x1=386, num3x3=386, chandim=-1,input_shape=input_shape)
    # 平均池化7*7,1V
    x = avg_pool_module(x=x, pool_size=(7, 7), strides=(1, 1), padding='valid')
    # dropout 40%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='MaM_Model')
    return model

In [222]:
mam_model_4_1 = self_def_mam_model_4(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)
mam_model_4_1.compile(optimizer="Adadelta",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])
batch_size = 16
gc.collect()


4128

In [229]:
mam_model_4_1.fit(X_train, y_train, epochs=3)

Train on 51573 samples
Epoch 1/3
22080/51573 [===========>..................] - ETA: 3:29 - loss: 5.7702 - accuracy: 0.0044

KeyboardInterrupt: 

In [ ]:
mam_model_4_2 = self_def_mam_model_4(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)
mam_model_4_2.compile(optimizer="Adam",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])
batch_size = 16
gc.collect()
mam_model_4_2.fit(X_train, y_train, epochs=3)

In [ ]:
batch_size = 16
mam_model_4_3 = self_def_mam_model_4(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)
mam_model_4_3.compile(optimizer="rmsprop",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])

gc.collect()
mam_model_4_3.fit(X_train, y_train, epochs=2)

In [230]:
gc.collect()

389

In [23]:
X_train[8][100]

NameError: name 'X_train' is not defined

In [244]:
#数据存在问题，需要对数据进行重新整理


### 重新加载图片数据

In [8]:
# 使用opencv 
import cv2 as cv
import os
import gc
import numpy as np

In [5]:
img_path = 'cardata\\'

In [6]:
img=cv.imread(img_path+'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_aWg.jpg',1)
img

array([[[140, 110,  99],
        [142, 112, 101],
        [144, 115, 101],
        ...,
        [145, 146, 174],
        [145, 143, 179],
        [ 98,  95, 134]],

       [[138, 108,  97],
        [140, 110,  99],
        [142, 113,  99],
        ...,
        [146, 147, 175],
        [141, 140, 174],
        [ 82,  80, 116]],

       [[136, 106,  95],
        [137, 107,  96],
        [139, 110,  96],
        ...,
        [146, 145, 171],
        [140, 139, 171],
        [ 82,  81, 115]],

       ...,

       [[180, 148, 142],
        [186, 154, 148],
        [190, 158, 152],
        ...,
        [227, 228, 232],
        [226, 227, 231],
        [224, 225, 229]],

       [[180, 150, 145],
        [182, 153, 146],
        [183, 154, 147],
        ...,
        [228, 227, 231],
        [226, 225, 229],
        [224, 223, 227]],

       [[183, 154, 149],
        [176, 147, 142],
        [170, 141, 134],
        ...,
        [222, 221, 225],
        [220, 219, 223],
        [219, 218, 222]]

In [7]:
cv.imshow('img', img)
cv.waitKey(0)

-1

In [8]:
# cv.destroyAllWindows()
# cv.waitKey(1)

In [9]:
imgs = os.listdir(img_path)
imgs

['Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_aWg.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_Bbw.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_Cvl.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_eeu.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_gQW.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_Gzs.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_kMx.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_Mro.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_Ofi.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_onJ.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_plr.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_RuR.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_SLr.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_UMh.jpg',
 'Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_Wxn.j

In [12]:
from tensorflow.keras.layers import concatenate


def load_img(n):

    # os.listdir(filename)返回filename中所有文件的文件名列表
    imgs = os.listdir(img_path)
    num = len(imgs)
#     print(num)
    label_make = [""for j in range(num)]  # Make
    label_module = [""for j in range(num)]  # Module
    label=[]
    data = []
    for i in range(min(num,n)):
        img = cv.imread(img_path+imgs[i], 1)
        data.append(img)
        label_make[i], label_module[i] = imgs[i].split('_')[0:2]

        if i % 1000 == 0:
            print(i/646, "%")
        del img
        gc.collect()
    for i in range(min(num,n)):
        label.append(label_make[i]+'_'+label_module[i])
    return data, label,label_make,label_module

In [13]:
gc.collect()
data, label,lanel_make,label_module = load_img(1000)

0.0 %


In [15]:
len(data)

1000

In [22]:
data[8]

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [19]:
img = cv.imread(img_path+imgs[8], 1)
cv.imshow('img', img)
cv.waitKey(0)

-1

In [24]:
# 经查勘，图片本身没有问题，因为白色背景所以有那么多255

### 使用更少的分类

#### 加载图片和标签

In [1]:
# 加载data、label
import scipy.io as io
data_mat_path = 'mats/'
label_mat_path = 'labels/'

data_filename = 'data_old.mat'
label_filename = 'label_old.mat'
data = io.loadmat(data_mat_path+data_filename)["name"]
label = io.loadmat(label_mat_path+label_filename)["name"]

In [2]:
data.shape

(64467, 60, 80, 3)

In [3]:
label.shape

(64467, 16)

#### 打乱顺序

In [10]:
import numpy as np
# list转np.array
# data = np.array(data)
label = np.array(label)
# 打乱顺序
# 读取data矩阵的第一维数（图片的个数）
num_example = len(data)
# 产生一个num_example范围，步长为1的序列
arr = np.arange(num_example)
print(arr)
# 调用函数，打乱顺序
np.random.shuffle(arr)
print(arr)
# 按照打乱的顺序，重新排序
data = data[arr]
label = label[arr]

[    0     1     2 ... 64464 64465 64466]
[  630 11584  3874 ... 49789 23668 53010]


In [11]:
gc.collect()

50

#### 生成各标签

In [13]:
label[9].shape

(16,)

In [17]:
cols_list = ['Make', 'Module', 'Year', 'MSRP', 'FrontWheelSize', 'Horsepower',
             'Displacement', 'EngineType', 'Width', 'Heigh', 'Length', 'GasMileage',
             'Drivetrain', 'PassengerCpacity', 'PassengerDoors', 'BodyStyle']
row = len(data)
l_make = []
l_module = []
l_year = []
l_msrp = []
l_fws = []
l_horsepower = []
l_dispm = []
l_enginetype = []
l_width = []
l_height = []
l_length = []
l_gas = []
l_drivetrain = []
l_pc = []
l_pd = []
l_bodystyle = []
for i in range(row):

    l_make.append(label[i][0].strip())
    l_module.append(label[i][1].strip())
    l_year.append(label[i][2].strip())
    l_msrp.append(label[i][3].strip())
    l_fws.append(label[i][4].strip())
    l_horsepower.append(label[i][5].strip())
    l_dispm.append(label[i][6].strip())
    l_enginetype.append(label[i][7].strip())
    l_width.append(label[i][8].strip())
    l_height.append(label[i][9].strip())
    l_length.append(label[i][10].strip())
    l_gas.append(label[i][11].strip())
    l_drivetrain.append(label[i][12].strip())
    l_pc.append(label[i][13].strip())
    l_pd.append(label[i][14].strip())
    l_bodystyle.append(label[i][15].strip())

In [19]:
l_make[0]

'Chevrolet'

#### 均值填充

#### 只对Make进行分类

In [63]:
# len(l_make)
# n=0
# for i in range(row):
#     print(l_make[i])


##### 独热编码

In [20]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_make = encoder.fit_transform(l_make)

In [22]:
y_make[1000]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

##### 划分训练集、测试集

In [21]:
r_train = 0.8


s_train = np.int(num_example * r_train)


X_make_train = data[:s_train]
y_make_train = y_make[:s_train]

X_make_test = data[s_train:]
y_make_test = y_make[s_train:]
#归一化
#归一化
X_make_train=X_make_train/255
X_make_test=X_make_test/255

In [23]:
#归一化
#归一化
X_make_train=X_make_train/255
X_make_test=X_make_test/255

##### 编译神经网络

In [28]:
batch_size=16
classes=y_make.shape[1]
height,width,channel=X_make_train[0].shape

make_model=self_def_mam_model(batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)

make_model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

##### 训练模型

In [29]:
make_model.fit(X_make_train, y_make_train, epochs=5)


Train on 51573 samples
Epoch 1/5
51573/51573 [==============================] - 370s 7ms/sample - loss: 3.4529 - accuracy: 0.0775
Epoch 2/5
51573/51573 [==============================] - 360s 7ms/sample - loss: 3.4105 - accuracy: 0.0795
Epoch 3/5
51573/51573 [==============================] - 359s 7ms/sample - loss: 3.4096 - accuracy: 0.0794
Epoch 4/5
51573/51573 [==============================] - 355s 7ms/sample - loss: 3.4078 - accuracy: 0.0794
Epoch 5/5
51573/51573 [==============================] - 354s 7ms/sample - loss: 3.4066 - accuracy: 0.0794


In [ ]:
# mam_model.evaluate(X_make_test, y_make_test)

##### 使用更深的网络

In [56]:
# 卷积层


def conv_module(x, filters,  strides, kernel_size, padding="same", activation='relu'):
    # conv-bn-relu
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               activation=activation,
               padding=padding
               )(x)
    return x

# 初始化层


def inception_module(x, num1x1_1, num1x1_2, num1x1_3, num1x1_4, num3x3, num_5x5, chandim=-1):
    #
    conv_1x1_1 = conv_module(x=x,
                             filters=num1x1_1,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    conv_1x1_2 = conv_module(x=x,
                             filters=num1x1_2,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    #
    conv_3_3 = conv_module(x=conv_1x1_2,
                           filters=num3x3,
                           kernel_size=(3, 3),
                           strides=(1, 1),
                           #                            input_shape=input_shape
                           )
    conv_1x1_3 = conv_module(x=x,
                             filters=num1x1_3,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )
    #
    conv_5_5 = conv_module(x=conv_1x1_3,
                           filters=num_5x5,
                           kernel_size=(5, 5),
                           strides=(1, 1),
                           #                            input_shape=input_shape
                           )
    max_pool = max_pool_module(x=x, pool_size=(
        3, 3), strides=(1, 1), padding='same')
    #
    conv_1x1_4 = conv_module(x=max_pool,
                             filters=num1x1_4,
                             kernel_size=(1, 1),
                             strides=(1, 1),
                             #                              input_shape=input_shape
                             )

    x = concatenate([conv_1x1_1, conv_3_3, conv_5_5, conv_1x1_4], axis=chandim)
    return x

# 最大值池化层


def max_pool_module(x, pool_size, strides,  padding='same'):
    x = MaxPooling2D(pool_size=pool_size,
                     strides=strides,
                     padding=padding,
                     # filters=filters
                     )(x)
    return x

# 平均值池化层


def avg_pool_module(x, pool_size, strides, padding='valid'):
    x = AveragePooling2D(pool_size=pool_size,
                         strides=strides,
                         padding=padding,
                         # filters=filters
                         )(x)
    return x


def self_def_make_model(batch_size, width, height, channel, classes):

    input_shape = (height, width, channel)
    chandim = -1
   # 定义神经网络各层
    # inputs = Input(shape=inputshape)

    # 输入层
    inputs = Input(shape=input_shape)
    # 卷积层 64，7*7,2S
    x = conv_module(x=inputs, filters=64,
                    strides=(2, 2), kernel_size=(7, 7))
    # 最大池化层64，3*3，2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))
    # 局部响应归一化
#     x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 卷积层 64，1*1，1S
    x = conv_module(x=inputs, filters=64,
                    strides=(1, 1), kernel_size=(1, 1))
    # 卷积层192,3*3,1S
    x = conv_module(x=inputs, filters=192,
                    strides=(1, 1), kernel_size=(3, 3))
   
    # 局部响应归一化
#     x = tf.nn.lrn(x, depth_radius=5, bias=1, alpha=1, beta=0.5, name=None)
    # 最大池化层192，(3,3)，2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1_1=64, num1x1_2=96, num1x1_3=16, num1x1_4=32,
                         num3x3=128, num_5x5=32)
    # 初始化层
    x = inception_module(x=x, num1x1_1=128, num1x1_2=128, num1x1_3=32, num1x1_4=64,
                         num3x3=192, num_5x5=96, chandim=-1)
    # 最大池化层,480,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3),  strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1_1=192, num1x1_2=96, num1x1_3=16, num1x1_4=64,
                         num3x3=208, num_5x5=48, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=160, num1x1_2=112, num1x1_3=24, num1x1_4=64,
                         num3x3=224, num_5x5=64, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=128, num1x1_2=128, num1x1_3=24, num1x1_4=64,
                         num3x3=256, num_5x5=64, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=112, num1x1_2=144, num1x1_3=32, num1x1_4=64, num3x3=288, num_5x5=64,
                         chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=256, num1x1_2=160, num1x1_3=32, num1x1_4=128,
                         num3x3=320, num_5x5=128, chandim=-1)
    # 最大池化层,832,3*3,2S
    x = max_pool_module(x=x, pool_size=(3, 3), strides=(2, 2))
    # 初始化层
    x = inception_module(x=x, num1x1_1=64, num1x1_2=96, num1x1_3=16, num1x1_4=32,
                         num3x3=128, num_5x5=32, chandim=-1)
    # 初始化层
    x = inception_module(x=x, num1x1_1=64, num1x1_2=96, num1x1_3=16, num1x1_4=32,
                         num3x3=128, num_5x5=32, chandim=-1)
 
    # 平均池化1024,7*7,1V
    x = avg_pool_module(x=x, pool_size=(7, 7), strides=(1, 1),
                         padding='valid')
    # dropout 40%
    x = Dropout(0.5)(x)
    # 全链接
    x = Flatten()(x)
    x = Dense(classes)(x)

    # softmax
    x = Activation('softmax')(x)

    # 构造model
    model = Model(inputs, x, name='Make_Model')
    return model

In [48]:
batch_size = 32
classes = y_make.shape[1]
height, width, channel = X_make_train[0].shape

make_model = self_def_make_model(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)

make_model.compile(optimizer="adam",
                   loss="categorical_crossentropy",
                   metrics=['accuracy'])

In [49]:
make_model.fit(X_make_train, y_make_train, epochs=1)

Train on 51573 samples
51573/51573 [==============================] - 218s 4ms/sample - loss: 3.4223 - accuracy: 0.0730


In [50]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
A = []
for i in range(30):
    a = make_model.fit(X_make_train, y_make_train, epochs=1)
    b = a.history['loss'][0]
    A.append(a)

Train on 51573 samples
51573/51573 [==============================] - 235s 5ms/sample - loss: 3.4130 - accuracy: 0.0763
Train on 51573 samples
51573/51573 [==============================] - 222s 4ms/sample - loss: 3.4105 - accuracy: 0.0775
Train on 51573 samples
51573/51573 [==============================] - 218s 4ms/sample - loss: 3.4100 - accuracy: 0.0773
Train on 51573 samples
51573/51573 [==============================] - 241s 5ms/sample - loss: 3.4096 - accuracy: 0.0780
Train on 51573 samples
51573/51573 [==============================] - 237s 5ms/sample - loss: 3.4098 - accuracy: 0.0781
Train on 51573 samples
51573/51573 [==============================] - 226s 4ms/sample - loss: 3.4094 - accuracy: 0.0785
Train on 51573 samples
51573/51573 [==============================] - 247s 5ms/sample - loss: 3.4092 - accuracy: 0.0775
Train on 51573 samples
51573/51573 [==============================] - 224s 4ms/sample - loss: 3.4090 - accuracy: 0.0782
Train on 51573 samples
51573/51573 [====

In [53]:
# import matplotlib.pyplot as plt
# A = np.array(A)
# B = np.array(range(len(A)))
# plt.plot(B, A, c='g')
# plt.show()

In [57]:
# 模型不收敛，尝试调大batchsize
batch_size = 64
classes = y_make.shape[1]
height, width, channel = X_make_train[0].shape

make_model = self_def_make_model(
    batch_size=batch_size, width=width, height=height, channel=channel, classes=classes)

make_model.compile(optimizer="adam",
                   loss="categorical_crossentropy",
                   metrics=['accuracy'])

In [58]:
make_model.fit(X_make_train, y_make_train, epochs=1)

Train on 51573 samples
51573/51573 [==============================] - 965s 19ms/sample - loss: 3.4197 - accuracy: 0.0768


#### 对BodyStyle进行分类

In [119]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_bs = encoder.fit_transform(l_bodystyle)

In [1]:
y_bs.shape

NameError: name 'y_bs' is not defined

In [122]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
X_bs_train, X_bs_test, y_bs_train, y_bs_test = train_test_split(
    data, y_bs, test_size=0.3)

In [123]:
X_bs_train.shape

(45126, 60, 80, 3)

In [124]:
 y_bs_test.shape

(19341, 9)

In [127]:
# 归一化
X_bs_train=X_bs_train/255
X_bs_test=X_bs_test/255

In [128]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [130]:
input_shape = (height, width, channel)

In [131]:
input_shape

(60, 80, 3)

In [132]:
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(7, 7),
                          strides=(3, 3),
                          activation='relu',
                          padding='same'
                          )
                   )

In [133]:
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
# bodyStyleModel.add(Dropout(0.2))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
# bodyStyleModel.add(Dropout(0.4))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))
bodyStyleModel.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_575 (Conv2D)          (None, 20, 27, 32)        4736      
_________________________________________________________________
conv2d_576 (Conv2D)          (None, 10, 14, 32)        25632     
_________________________________________________________________
conv2d_577 (Conv2D)          (None, 10, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 5, 7, 32)          0         
_________________________________________________________________
conv2d_578 (Conv2D)          (None, 5, 7, 64)          51264     
_________________________________________________________________
conv2d_579 (Conv2D)          (None, 3, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_123 (MaxPoolin (None, 1, 2, 64)          0

In [134]:
from tensorflow.keras.optimizers import SGD

learning_rate = 0.01
decay = 1e-6
momentum = 0.8
nesterov = True
sgd_optimizer = SGD(lr=learning_rate, decay=decay,
                    momentum=momentum, nesterov=nesterov)

In [112]:
# from tensorflow.keras.optimizers import Adam
# opt = Adam(lr=0.001, decay=1e-6)

In [135]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )

In [137]:
batch_size = 20  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 16s 364us/sample - loss: 1.4577 - accuracy: 0.4296
Epoch 2/10
45126/45126 [==============================] - 14s 317us/sample - loss: 1.2346 - accuracy: 0.5258
Epoch 3/10
45126/45126 [==============================] - 14s 316us/sample - loss: 1.1366 - accuracy: 0.5659
Epoch 4/10
45126/45126 [==============================] - 14s 317us/sample - loss: 1.0665 - accuracy: 0.5948
Epoch 5/10
45126/45126 [==============================] - 14s 318us/sample - loss: 1.0067 - accuracy: 0.6222
Epoch 6/10
45126/45126 [==============================] - 14s 316us/sample - loss: 0.9515 - accuracy: 0.6422
Epoch 7/10
45126/45126 [==============================] - 14s 316us/sample - loss: 0.9048 - accuracy: 0.6623
Epoch 8/10
45126/45126 [==============================] - 14s 314us/sample - loss: 0.8609 - accuracy: 0.6779
Epoch 9/10
45126/45126 [==============================] - 14s 315us/sample - loss: 0.8207 - accuracy: 0.6

In [139]:
batch_size = 25  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 12s 276us/sample - loss: 0.6835 - accuracy: 0.7517
Epoch 2/10
45126/45126 [==============================] - 12s 269us/sample - loss: 0.6121 - accuracy: 0.7773
Epoch 3/10
45126/45126 [==============================] - 12s 268us/sample - loss: 0.5878 - accuracy: 0.7869
Epoch 4/10
45126/45126 [==============================] - 12s 268us/sample - loss: 0.5651 - accuracy: 0.7961
Epoch 5/10
45126/45126 [==============================] - 12s 268us/sample - loss: 0.5522 - accuracy: 0.8015
Epoch 6/10
45126/45126 [==============================] - 12s 269us/sample - loss: 0.5384 - accuracy: 0.8064
Epoch 7/10
45126/45126 [==============================] - 12s 268us/sample - loss: 0.5177 - accuracy: 0.8157
Epoch 8/10
45126/45126 [==============================] - 12s 267us/sample - loss: 0.5138 - accuracy: 0.8165
Epoch 9/10
45126/45126 [==============================] - 12s 268us/sample - loss: 0.5125 - accuracy: 0.8

In [142]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))
#测试集上正确率只有0.6

[1.4555066510542651, 0.6142909]


In [143]:
# MODEL_PATH = 'model\\BodyStyleModel.h5'
# face_recognition_model.save(MODEL_PATH)

In [144]:
batch_size = 25  # 每批训练数据量的大小
epochs = 50
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/50
45126/45126 [==============================] - 12s 274us/sample - loss: 0.5034 - accuracy: 0.8213
Epoch 2/50
45126/45126 [==============================] - 12s 272us/sample - loss: 0.4993 - accuracy: 0.8261
Epoch 3/50
45126/45126 [==============================] - 12s 271us/sample - loss: 0.5149 - accuracy: 0.8196
Epoch 4/50
45126/45126 [==============================] - 12s 272us/sample - loss: 0.5066 - accuracy: 0.8229
Epoch 5/50
45126/45126 [==============================] - 12s 271us/sample - loss: 0.5085 - accuracy: 0.8229
Epoch 6/50
45126/45126 [==============================] - 12s 270us/sample - loss: 0.5027 - accuracy: 0.8256
Epoch 7/50
45126/45126 [==============================] - 12s 270us/sample - loss: 0.5114 - accuracy: 0.8247
Epoch 8/50
45126/45126 [==============================] - 12s 271us/sample - loss: 0.5104 - accuracy: 0.8228
Epoch 9/50
45126/45126 [==============================] - 12s 272us/sample - loss: 0.5060 - accuracy: 0.8

In [145]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.5618455069898776, 0.57494444]


In [201]:
# 模型复杂化，提高精确率
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.2))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
# bodyStyleModel.add(Dropout(0.4))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))
bodyStyleModel.summary()


Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_727 (Conv2D)          (None, 60, 80, 32)        896       
_________________________________________________________________
conv2d_728 (Conv2D)          (None, 30, 40, 32)        25632     
_________________________________________________________________
conv2d_729 (Conv2D)          (None, 30, 40, 32)        9248      
_________________________________________________________________
max_pooling2d_156 (MaxPoolin (None, 15, 20, 32)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 15, 20, 32)        0         
_________________________________________________________________
conv2d_730 (Conv2D)          (None, 15, 20, 64)        51264     
_________________________________________________________________
conv2d_731 (Conv2D)          (None, 8, 10, 64)       

In [202]:
learning_rate = 0.01
decay = 1e-6
momentum = 0.8
nesterov = True
sgd_optimizer = SGD(lr=learning_rate, decay=decay,
                    momentum=momentum, nesterov=nesterov)

In [203]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )

In [204]:
batch_size = 32  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 20s 432us/sample - loss: 1.5236 - accuracy: 0.4019
Epoch 2/10
45126/45126 [==============================] - 19s 412us/sample - loss: 1.2810 - accuracy: 0.5105
Epoch 3/10
45126/45126 [==============================] - 19s 415us/sample - loss: 1.1539 - accuracy: 0.5624
Epoch 4/10
45126/45126 [==============================] - 19s 417us/sample - loss: 1.0587 - accuracy: 0.6033
Epoch 5/10
45126/45126 [==============================] - 19s 416us/sample - loss: 0.9594 - accuracy: 0.6410
Epoch 6/10
45126/45126 [==============================] - 19s 417us/sample - loss: 0.8643 - accuracy: 0.6835
Epoch 7/10
45126/45126 [==============================] - 19s 424us/sample - loss: 0.7591 - accuracy: 0.7243
Epoch 8/10
45126/45126 [==============================] - 19s 417us/sample - loss: 0.6626 - accuracy: 0.7604
Epoch 9/10
45126/45126 [==============================] - 19s 417us/sample - loss: 0.5623 - accuracy: 0.7

In [205]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.1809007015009343, 0.65658444]


In [207]:
batch_size = 26  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 21s 463us/sample - loss: 0.4762 - accuracy: 0.8327
Epoch 2/10
45126/45126 [==============================] - 21s 463us/sample - loss: 0.3725 - accuracy: 0.8715
Epoch 3/10
45126/45126 [==============================] - 21s 463us/sample - loss: 0.3220 - accuracy: 0.8921
Epoch 4/10
45126/45126 [==============================] - 21s 465us/sample - loss: 0.2731 - accuracy: 0.9104
Epoch 5/10
45126/45126 [==============================] - 21s 466us/sample - loss: 0.2420 - accuracy: 0.9233
Epoch 6/10
45126/45126 [==============================] - 21s 466us/sample - loss: 0.2127 - accuracy: 0.9329
Epoch 7/10
45126/45126 [==============================] - 21s 464us/sample - loss: 0.1933 - accuracy: 0.9375
Epoch 8/10
45126/45126 [==============================] - 21s 465us/sample - loss: 0.1785 - accuracy: 0.9438
Epoch 9/10
45126/45126 [==============================] - 21s 465us/sample - loss: 0.1638 - accuracy: 0.9

In [208]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.6874950016908885, 0.69458663]


In [209]:
# 训练集上准确率0.95，测试集只有0.69，出现过拟合


In [210]:
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.2))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.2))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))
bodyStyleModel.summary()


Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_732 (Conv2D)          (None, 60, 80, 32)        896       
_________________________________________________________________
conv2d_733 (Conv2D)          (None, 30, 40, 32)        25632     
_________________________________________________________________
conv2d_734 (Conv2D)          (None, 30, 40, 32)        9248      
_________________________________________________________________
max_pooling2d_158 (MaxPoolin (None, 15, 20, 32)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 15, 20, 32)        0         
_________________________________________________________________
conv2d_735 (Conv2D)          (None, 15, 20, 64)        51264     
_________________________________________________________________
conv2d_736 (Conv2D)          (None, 8, 10, 64)       

In [211]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )

In [212]:
batch_size = 32  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 20s 436us/sample - loss: 1.5618 - accuracy: 0.3864
Epoch 2/10
45126/45126 [==============================] - 19s 420us/sample - loss: 1.3385 - accuracy: 0.4897
Epoch 3/10
45126/45126 [==============================] - 19s 422us/sample - loss: 1.2122 - accuracy: 0.5360
Epoch 4/10
45126/45126 [==============================] - 19s 427us/sample - loss: 1.1269 - accuracy: 0.5713
Epoch 5/10
45126/45126 [==============================] - 19s 428us/sample - loss: 1.0523 - accuracy: 0.6022
Epoch 6/10
45126/45126 [==============================] - 19s 425us/sample - loss: 0.9794 - accuracy: 0.6315
Epoch 7/10
45126/45126 [==============================] - 19s 425us/sample - loss: 0.9066 - accuracy: 0.6630
Epoch 8/10
45126/45126 [==============================] - 19s 424us/sample - loss: 0.8384 - accuracy: 0.6900
Epoch 9/10
45126/45126 [==============================] - 19s 424us/sample - loss: 0.7658 - accuracy: 0.7

In [213]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.0285654090736678, 0.6299054]


In [215]:
batch_size = 32  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 19s 421us/sample - loss: 0.6373 - accuracy: 0.7712
Epoch 2/10
45126/45126 [==============================] - 19s 426us/sample - loss: 0.5578 - accuracy: 0.7989
Epoch 3/10
45126/45126 [==============================] - 19s 424us/sample - loss: 0.4958 - accuracy: 0.8246
Epoch 4/10
45126/45126 [==============================] - 19s 424us/sample - loss: 0.4512 - accuracy: 0.8395
Epoch 5/10
45126/45126 [==============================] - 19s 423us/sample - loss: 0.4206 - accuracy: 0.8523
Epoch 6/10
45126/45126 [==============================] - 19s 423us/sample - loss: 0.3770 - accuracy: 0.8669
Epoch 7/10
45126/45126 [==============================] - 19s 423us/sample - loss: 0.3548 - accuracy: 0.8760
Epoch 8/10
45126/45126 [==============================] - 19s 424us/sample - loss: 0.3295 - accuracy: 0.8853
Epoch 9/10
45126/45126 [==============================] - 19s 425us/sample - loss: 0.3016 - accuracy: 0.8

In [216]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.2456707448872075, 0.68920946]


In [218]:
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.2))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))



In [219]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )

In [220]:
batch_size = 32  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 20s 438us/sample - loss: 1.6020 - accuracy: 0.3590
Epoch 2/10
45126/45126 [==============================] - 19s 421us/sample - loss: 1.3808 - accuracy: 0.4674
Epoch 3/10
45126/45126 [==============================] - 19s 421us/sample - loss: 1.2498 - accuracy: 0.5208
Epoch 4/10
45126/45126 [==============================] - 19s 425us/sample - loss: 1.1636 - accuracy: 0.5554
Epoch 5/10
45126/45126 [==============================] - 19s 424us/sample - loss: 1.0924 - accuracy: 0.5854
Epoch 6/10
45126/45126 [==============================] - 19s 424us/sample - loss: 1.0279 - accuracy: 0.6118
Epoch 7/10
45126/45126 [==============================] - 19s 425us/sample - loss: 0.9592 - accuracy: 0.6431
Epoch 8/10
45126/45126 [==============================] - 19s 425us/sample - loss: 0.8969 - accuracy: 0.6647
Epoch 9/10
45126/45126 [==============================] - 19s 425us/sample - loss: 0.8306 - accuracy: 0.6

In [221]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[0.9641221859395815, 0.64614034]


In [223]:
batch_size = 40  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 18s 395us/sample - loss: 0.6338 - accuracy: 0.7693
Epoch 2/10
45126/45126 [==============================] - 18s 389us/sample - loss: 0.5679 - accuracy: 0.7943
Epoch 3/10
45126/45126 [==============================] - 18s 392us/sample - loss: 0.5288 - accuracy: 0.8098
Epoch 4/10
45126/45126 [==============================] - 18s 392us/sample - loss: 0.4816 - accuracy: 0.8250
Epoch 5/10
45126/45126 [==============================] - 18s 394us/sample - loss: 0.4556 - accuracy: 0.8358
Epoch 6/10
45126/45126 [==============================] - 18s 394us/sample - loss: 0.4236 - accuracy: 0.8486
Epoch 7/10
45126/45126 [==============================] - 18s 393us/sample - loss: 0.3909 - accuracy: 0.8629
Epoch 8/10
45126/45126 [==============================] - 18s 393us/sample - loss: 0.3598 - accuracy: 0.8745
Epoch 9/10
45126/45126 [==============================] - 18s 393us/sample - loss: 0.3445 - accuracy: 0.8

In [224]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.1620387726772856, 0.69060546]


In [225]:
batch_size = 40  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 18s 390us/sample - loss: 0.3108 - accuracy: 0.8913
Epoch 2/10
45126/45126 [==============================] - 18s 389us/sample - loss: 0.2882 - accuracy: 0.9012
Epoch 3/10
45126/45126 [==============================] - 18s 392us/sample - loss: 0.2781 - accuracy: 0.9046
Epoch 4/10
45126/45126 [==============================] - 18s 392us/sample - loss: 0.2726 - accuracy: 0.9066
Epoch 5/10
45126/45126 [==============================] - 18s 392us/sample - loss: 0.2452 - accuracy: 0.9154
Epoch 6/10
45126/45126 [==============================] - 18s 394us/sample - loss: 0.2352 - accuracy: 0.9192
Epoch 7/10
45126/45126 [==============================] - 18s 393us/sample - loss: 0.2270 - accuracy: 0.9203
Epoch 8/10
45126/45126 [==============================] - 18s 393us/sample - loss: 0.2222 - accuracy: 0.9236
Epoch 9/10
45126/45126 [==============================] - 18s 392us/sample - loss: 0.2084 - accuracy: 0.9

In [226]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.3027688710101912, 0.7050825]


In [228]:
# 减小复杂度解决过拟合
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )

bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.2))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))



In [229]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )
batch_size = 40  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 17s 370us/sample - loss: 1.5851 - accuracy: 0.3746
Epoch 2/10
45126/45126 [==============================] - 16s 354us/sample - loss: 1.3677 - accuracy: 0.4794
Epoch 3/10
45126/45126 [==============================] - 16s 357us/sample - loss: 1.2432 - accuracy: 0.5285
Epoch 4/10
45126/45126 [==============================] - 16s 360us/sample - loss: 1.1620 - accuracy: 0.5601
Epoch 5/10
45126/45126 [==============================] - 16s 358us/sample - loss: 1.0948 - accuracy: 0.5878
Epoch 6/10
45126/45126 [==============================] - 16s 356us/sample - loss: 1.0341 - accuracy: 0.6119
Epoch 7/10
45126/45126 [==============================] - 16s 358us/sample - loss: 0.9737 - accuracy: 0.6365
Epoch 8/10
45126/45126 [==============================] - 16s 359us/sample - loss: 0.9195 - accuracy: 0.6580- loss: 0.9197 - accu
Epoch 9/10
45126/45126 [==============================] - 16s 360us/sample - loss: 0

In [230]:
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

[1.005734764600785, 0.64360684]


In [231]:
batch_size = 40  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 16s 358us/sample - loss: 0.7446 - accuracy: 0.7266
Epoch 2/10
45126/45126 [==============================] - 16s 362us/sample - loss: 0.6904 - accuracy: 0.7493
Epoch 3/10
45126/45126 [==============================] - 16s 360us/sample - loss: 0.6351 - accuracy: 0.7689
Epoch 4/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.5900 - accuracy: 0.7878
Epoch 5/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.5470 - accuracy: 0.8024
Epoch 6/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.5037 - accuracy: 0.8171
Epoch 7/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.4695 - accuracy: 0.8320
Epoch 8/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.4400 - accuracy: 0.8430
Epoch 9/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.4053 - accuracy: 0.8

In [232]:
batch_size = 40  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))


Train on 45126 samples
Epoch 1/10
45126/45126 [==============================] - 16s 355us/sample - loss: 0.3551 - accuracy: 0.8750
Epoch 2/10
45126/45126 [==============================] - 16s 357us/sample - loss: 0.3436 - accuracy: 0.8790
Epoch 3/10
45126/45126 [==============================] - 16s 358us/sample - loss: 0.3167 - accuracy: 0.8900
Epoch 4/10
45126/45126 [==============================] - 16s 358us/sample - loss: 0.3036 - accuracy: 0.8950
Epoch 5/10
45126/45126 [==============================] - 16s 359us/sample - loss: 0.2832 - accuracy: 0.9012- loss: 0.2805 
Epoch 6/10
45126/45126 [==============================] - 16s 360us/sample - loss: 0.2732 - accuracy: 0.9062
Epoch 7/10
45126/45126 [==============================] - 16s 360us/sample - loss: 0.2614 - accuracy: 0.9092
Epoch 8/10
45126/45126 [==============================] - 16s 361us/sample - loss: 0.2529 - accuracy: 0.9119
Epoch 9/10
45126/45126 [==============================] - 16s 360us/sample - loss: 0.2445 

In [233]:
batch_size = 40  # 每批训练数据量的大小
epochs = 20
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/20
45126/45126 [==============================] - 16s 355us/sample - loss: 0.2248 - accuracy: 0.9236
Epoch 2/20
45126/45126 [==============================] - 16s 355us/sample - loss: 0.2174 - accuracy: 0.9269
Epoch 3/20
45126/45126 [==============================] - 16s 356us/sample - loss: 0.2062 - accuracy: 0.9299
Epoch 4/20
45126/45126 [==============================] - 16s 357us/sample - loss: 0.2030 - accuracy: 0.9320
Epoch 5/20
45126/45126 [==============================] - 16s 358us/sample - loss: 0.1937 - accuracy: 0.9336
Epoch 6/20
45126/45126 [==============================] - 16s 363us/sample - loss: 0.1860 - accuracy: 0.9370
Epoch 7/20
45126/45126 [==============================] - 16s 359us/sample - loss: 0.1811 - accuracy: 0.9373
Epoch 8/20
45126/45126 [==============================] - 16s 360us/sample - loss: 0.1802 - accuracy: 0.9387
Epoch 9/20
45126/45126 [==============================] - 16s 359us/sample - loss: 0.1719 - accuracy: 0.9

In [234]:
# 增大dropout比例


In [235]:
# 减小复杂度解决过拟合
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )

bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.4))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))



In [236]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )
batch_size = 40  # 每批训练数据量的大小
epochs = 50
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/50
45126/45126 [==============================] - 17s 371us/sample - loss: 1.6217 - accuracy: 0.3496
Epoch 2/50
45126/45126 [==============================] - 16s 355us/sample - loss: 1.4809 - accuracy: 0.4315
Epoch 3/50
45126/45126 [==============================] - 16s 358us/sample - loss: 1.3296 - accuracy: 0.4936- loss: 1.330
Epoch 4/50
45126/45126 [==============================] - 16s 358us/sample - loss: 1.2449 - accuracy: 0.5281
Epoch 5/50
45126/45126 [==============================] - 16s 359us/sample - loss: 1.1883 - accuracy: 0.5463
Epoch 6/50
45126/45126 [==============================] - 16s 360us/sample - loss: 1.1355 - accuracy: 0.5697
Epoch 7/50
45126/45126 [==============================] - 16s 362us/sample - loss: 1.0906 - accuracy: 0.5879
Epoch 8/50
45126/45126 [==============================] - 16s 363us/sample - loss: 1.0491 - accuracy: 0.6077
Epoch 9/50
45126/45126 [==============================] - 16s 363us/sample - loss: 1.0080 - 

In [237]:
# 减小复杂度解决过拟合
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )

bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.5))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))



In [238]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )
batch_size = 50  # 每批训练数据量的大小
epochs = 30
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/30
45126/45126 [==============================] - 16s 349us/sample - loss: 1.6369 - accuracy: 0.3304
Epoch 2/30
45126/45126 [==============================] - 15s 333us/sample - loss: 1.5584 - accuracy: 0.3913
Epoch 3/30
45126/45126 [==============================] - 15s 327us/sample - loss: 1.4530 - accuracy: 0.4443
Epoch 4/30
45126/45126 [==============================] - 15s 325us/sample - loss: 1.3680 - accuracy: 0.4768
Epoch 5/30
45126/45126 [==============================] - 15s 328us/sample - loss: 1.3052 - accuracy: 0.4992
Epoch 6/30
45126/45126 [==============================] - 15s 328us/sample - loss: 1.2636 - accuracy: 0.5186
Epoch 7/30
45126/45126 [==============================] - 15s 330us/sample - loss: 1.2275 - accuracy: 0.5314
Epoch 8/30
45126/45126 [==============================] - 15s 331us/sample - loss: 1.1992 - accuracy: 0.5434
Epoch 9/30
45126/45126 [==============================] - 15s 328us/sample - loss: 1.1739 - accuracy: 0.5

In [240]:
batch_size = 50  # 每批训练数据量的大小
epochs = 20
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/20
45126/45126 [==============================] - 15s 328us/sample - loss: 0.8487 - accuracy: 0.6817
Epoch 2/20
45126/45126 [==============================] - 15s 327us/sample - loss: 0.8233 - accuracy: 0.6909
Epoch 3/20
45126/45126 [==============================] - 15s 330us/sample - loss: 0.8154 - accuracy: 0.6940
Epoch 4/20
45126/45126 [==============================] - 15s 330us/sample - loss: 0.8053 - accuracy: 0.6982
Epoch 5/20
45126/45126 [==============================] - 15s 331us/sample - loss: 0.7938 - accuracy: 0.7016- loss: 0.7939 - accura - ETA: 0s - loss:
Epoch 6/20
45126/45126 [==============================] - 15s 329us/sample - loss: 0.7869 - accuracy: 0.7050
Epoch 7/20
45126/45126 [==============================] - 15s 330us/sample - loss: 0.7747 - accuracy: 0.7079
Epoch 8/20
45126/45126 [==============================] - 15s 329us/sample - loss: 0.7631 - accuracy: 0.7138
Epoch 9/20
45126/45126 [==============================] - 15s 33

In [241]:
batch_size = 50  # 每批训练数据量的大小
epochs = 50
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/50
45126/45126 [==============================] - 15s 328us/sample - loss: 0.6820 - accuracy: 0.7462
Epoch 2/50
45126/45126 [==============================] - 15s 327us/sample - loss: 0.6658 - accuracy: 0.7512
Epoch 3/50
45126/45126 [==============================] - 15s 326us/sample - loss: 0.6623 - accuracy: 0.7523
Epoch 4/50
45126/45126 [==============================] - 15s 327us/sample - loss: 0.6572 - accuracy: 0.7546
Epoch 5/50
45126/45126 [==============================] - 15s 328us/sample - loss: 0.6475 - accuracy: 0.7593
Epoch 6/50
45126/45126 [==============================] - 15s 329us/sample - loss: 0.6449 - accuracy: 0.7581
Epoch 7/50
45126/45126 [==============================] - 15s 327us/sample - loss: 0.6464 - accuracy: 0.7595
Epoch 8/50
45126/45126 [==============================] - 15s 330us/sample - loss: 0.6341 - accuracy: 0.7629
Epoch 9/50
45126/45126 [==============================] - 15s 331us/sample - loss: 0.6327 - accuracy: 0.7

KeyboardInterrupt: 

In [242]:
# 略显欠拟合

In [243]:
# 减小复杂度解决过拟合
# 增加dropout
# 构造网络

bodyStyleModel = Sequential()
# 第一层
bodyStyleModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
bodyStyleModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )

bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
bodyStyleModel.add(MaxPooling2D(pool_size=(2, 2)))
bodyStyleModel.add(Dropout(0.3))
bodyStyleModel.add(Flatten())
bodyStyleModel.add(Dense(512, activation='relu'))
bodyStyleModel.add(Dropout(0.4))
bodyStyleModel.add(Dense(len(y_bs[0]), activation='softmax'))



In [244]:
bodyStyleModel.compile(optimizer=sgd_optimizer,
                   loss="categorical_crossentropy",
                   metrics=['accuracy']
                   )
batch_size = 50  # 每批训练数据量的大小
epochs = 50
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/50
45126/45126 [==============================] - 16s 345us/sample - loss: 1.6401 - accuracy: 0.3255
Epoch 2/50
45126/45126 [==============================] - 15s 328us/sample - loss: 1.5808 - accuracy: 0.3705- loss: 1.5809 - accuracy: 0.37
Epoch 3/50
45126/45126 [==============================] - 15s 329us/sample - loss: 1.4856 - accuracy: 0.4271
Epoch 4/50
45126/45126 [==============================] - 15s 330us/sample - loss: 1.3875 - accuracy: 0.4682
Epoch 5/50
45126/45126 [==============================] - 15s 330us/sample - loss: 1.3225 - accuracy: 0.4937
Epoch 6/50
45126/45126 [==============================] - 15s 330us/sample - loss: 1.2724 - accuracy: 0.5141
Epoch 7/50
45126/45126 [==============================] - 15s 329us/sample - loss: 1.2324 - accuracy: 0.5315- loss: 1.2342 - accura
Epoch 8/50
45126/45126 [==============================] - 15s 330us/sample - loss: 1.2039 - accuracy: 0.5414
Epoch 9/50
45126/45126 [===========================

In [245]:

batch_size = 50  # 每批训练数据量的大小
epochs = 50
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/50
45126/45126 [==============================] - 15s 325us/sample - loss: 0.6204 - accuracy: 0.7699
Epoch 2/50
45126/45126 [==============================] - 15s 324us/sample - loss: 0.6132 - accuracy: 0.7732
Epoch 3/50
45126/45126 [==============================] - 15s 327us/sample - loss: 0.6045 - accuracy: 0.7760
Epoch 4/50
45126/45126 [==============================] - 15s 326us/sample - loss: 0.5955 - accuracy: 0.7792
Epoch 5/50
45126/45126 [==============================] - 15s 327us/sample - loss: 0.5929 - accuracy: 0.7795
Epoch 6/50
45126/45126 [==============================] - 15s 328us/sample - loss: 0.5890 - accuracy: 0.7815
Epoch 7/50
45126/45126 [==============================] - 15s 329us/sample - loss: 0.5772 - accuracy: 0.7868
Epoch 8/50
45126/45126 [==============================] - 15s 329us/sample - loss: 0.5722 - accuracy: 0.7890
Epoch 9/50
45126/45126 [==============================] - 15s 327us/sample - loss: 0.5748 - accuracy: 0.7

In [246]:

batch_size = 60  # 每批训练数据量的大小
epochs = 5
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 45126 samples
Epoch 1/5
45126/45126 [==============================] - 14s 312us/sample - loss: 0.4060 - accuracy: 0.8532
Epoch 2/5
45126/45126 [==============================] - 14s 306us/sample - loss: 0.3974 - accuracy: 0.8557
Epoch 3/5
45126/45126 [==============================] - 14s 313us/sample - loss: 0.3998 - accuracy: 0.8559
Epoch 4/5
45126/45126 [==============================] - 14s 312us/sample - loss: 0.3919 - accuracy: 0.8577
Epoch 5/5
45126/45126 [==============================] - 14s 313us/sample - loss: 0.3881 - accuracy: 0.8589
[0.8525082108634352, 0.72674626]


In [247]:
# 略显过拟合

In [248]:
# 接下来试试扩充一训练集


In [249]:
X_bs_train, X_bs_test, y_bs_train, y_bs_test = train_test_split(
    data, y_bs, test_size=0.1)
X_bs_train=X_bs_train/255
X_bs_test=X_bs_test/255

In [251]:
batch_size = 50  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 58020 samples
Epoch 1/10
58020/58020 [==============================] - 19s 330us/sample - loss: 0.5750 - accuracy: 0.8018
Epoch 2/10
58020/58020 [==============================] - 19s 331us/sample - loss: 0.5585 - accuracy: 0.8072
Epoch 3/10
58020/58020 [==============================] - 19s 331us/sample - loss: 0.5553 - accuracy: 0.8075
Epoch 4/10
58020/58020 [==============================] - 19s 331us/sample - loss: 0.5370 - accuracy: 0.8112
Epoch 5/10
58020/58020 [==============================] - 19s 329us/sample - loss: 0.5362 - accuracy: 0.8127
Epoch 6/10
58020/58020 [==============================] - 19s 331us/sample - loss: 0.5263 - accuracy: 0.8130
Epoch 7/10
58020/58020 [==============================] - 19s 330us/sample - loss: 0.5213 - accuracy: 0.8148
Epoch 8/10
58020/58020 [==============================] - 19s 330us/sample - loss: 0.5163 - accuracy: 0.8164
Epoch 9/10
58020/58020 [==============================] - 19s 330us/sample - loss: 0.5133 - accuracy: 0.8

In [255]:
batch_size = 50  # 每批训练数据量的大小
epochs = 10
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 58020 samples
Epoch 1/10
58020/58020 [==============================] - 19s 332us/sample - loss: 0.5146 - accuracy: 0.8177
Epoch 2/10
58020/58020 [==============================] - 19s 332us/sample - loss: 0.4931 - accuracy: 0.8211
Epoch 3/10
58020/58020 [==============================] - 20s 345us/sample - loss: 0.4936 - accuracy: 0.8221
Epoch 4/10
58020/58020 [==============================] - 22s 377us/sample - loss: 0.4808 - accuracy: 0.8272
Epoch 5/10
58020/58020 [==============================] - 19s 326us/sample - loss: 0.4805 - accuracy: 0.8253
Epoch 6/10
58020/58020 [==============================] - 19s 323us/sample - loss: 0.4715 - accuracy: 0.8283
Epoch 7/10
58020/58020 [==============================] - 19s 328us/sample - loss: 0.4716 - accuracy: 0.8282
Epoch 8/10
58020/58020 [==============================] - 19s 332us/sample - loss: 0.4679 - accuracy: 0.8307
Epoch 9/10
58020/58020 [==============================] - 19s 331us/sample - loss: 0.4656 - accuracy: 0.8

In [257]:
batch_size = 50  # 每批训练数据量的大小
epochs = 20
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 58020 samples
Epoch 1/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4367 - accuracy: 0.8419
Epoch 2/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4367 - accuracy: 0.8417
Epoch 3/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4258 - accuracy: 0.8464
Epoch 4/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4197 - accuracy: 0.8478
Epoch 5/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4229 - accuracy: 0.8471
Epoch 6/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4264 - accuracy: 0.8456s - loss: 0
Epoch 7/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4198 - accuracy: 0.8472
Epoch 8/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4208 - accuracy: 0.8471
Epoch 9/20
58020/58020 [==============================] - 75s 1ms/sample - loss: 0.4218 - accuracy: 0.8474
Epo

In [259]:
 batch_size = 50  # 每批训练数据量的大小
epochs = 20
bodyStyleModel.fit(X_bs_train,
                   y_bs_train,
                   epochs=epochs,
                   batch_size=batch_size,
                   shuffle=True,
                   #                    validation_data=(valid_input, valid_output)
                   )
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))
print(bodyStyleModel.evaluate(X_bs_test,y_bs_test,verbose=0))

Train on 58020 samples
Epoch 1/20
58020/58020 [==============================] - 25s 426us/sample - loss: 0.3721 - accuracy: 0.8665
Epoch 2/20
58020/58020 [==============================] - 30s 523us/sample - loss: 0.3703 - accuracy: 0.8656
Epoch 3/20
58020/58020 [==============================] - 30s 514us/sample - loss: 0.3689 - accuracy: 0.8675
Epoch 4/20
58020/58020 [==============================] - 23s 405us/sample - loss: 0.3664 - accuracy: 0.8680
Epoch 5/20
58020/58020 [==============================] - 27s 471us/sample - loss: 0.3714 - accuracy: 0.8667
Epoch 6/20
58020/58020 [==============================] - 30s 523us/sample - loss: 0.3647 - accuracy: 0.8674
Epoch 7/20
58020/58020 [==============================] - 31s 527us/sample - loss: 0.3646 - accuracy: 0.8692
Epoch 8/20
58020/58020 [==============================] - 31s 532us/sample - loss: 0.3628 - accuracy: 0.8688
Epoch 9/20
58020/58020 [==============================] - 31s 537us/sample - loss: 0.3692 - accuracy: 0.8

#### 保存BodyStyle模型

In [262]:
MODEL_PATH = 'model/bodystyle_model.h5'
bodyStyleModel.save(MODEL_PATH)

#### 只对make分类

In [263]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_make = encoder.fit_transform(l_make)
# 划分训练集测试集
from sklearn.model_selection import train_test_split
X_make_train, X_make_test, y_make_train, y_make_test = train_test_split(
    data, y_make, test_size=0.2)
# 归一化
X_make_train=X_make_train/255
X_make_test=X_make_test/255

In [264]:
# 减小复杂度解决过拟合
# 增加dropout
# 构造网络

makeModel = Sequential()
# 第一层
makeModel.add(Conv2D(filters=32,
                          input_shape=input_shape,
                          kernel_size=(3, 3),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
# 中间各层
makeModel.add(Conv2D(filters=32,
                          kernel_size=(5, 5),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )

makeModel.add(MaxPooling2D(pool_size=(2, 2)))
makeModel.add(Dropout(0.3))
makeModel.add(Conv2D(filters=64,
                          kernel_size=(5, 5),
                          strides=(1, 1),
                          activation='relu',
                          padding='same'
                          )
                   )
makeModel.add(Conv2D(filters=64,
                          kernel_size=(3, 3),
                          strides=(2, 2),
                          activation='relu',
                          padding='same'
                          )
                   )
makeModel.add(MaxPooling2D(pool_size=(2, 2)))
makeModel.add(Dropout(0.3))
makeModel.add(Flatten())
makeModel.add(Dense(512, activation='relu'))
makeModel.add(Dropout(0.4))
makeModel.add(Dense(len(y_make[0]), activation='softmax'))



In [265]:
makeModel.compile(optimizer=sgd_optimizer,
                  loss="categorical_crossentropy",
                  metrics=['accuracy']
                  )
batch_size = 50  # 每批训练数据量的大小
epochs = 100
makeModel.fit(X_make_train,
              y_make_train,
              epochs=epochs,
              batch_size=batch_size,
              shuffle=True,
              #                    validation_data=(valid_input, valid_output)
              )

Train on 51573 samples
Epoch 1/100
51573/51573 [==============================] - 20s 388us/sample - loss: 3.4360 - accuracy: 0.0723
Epoch 2/100
51573/51573 [==============================] - 18s 357us/sample - loss: 3.4080 - accuracy: 0.0763
Epoch 3/100
51573/51573 [==============================] - 18s 350us/sample - loss: 3.3963 - accuracy: 0.0797
Epoch 4/100
51573/51573 [==============================] - 18s 350us/sample - loss: 3.3779 - accuracy: 0.0876
Epoch 5/100
51573/51573 [==============================] - 18s 352us/sample - loss: 3.3539 - accuracy: 0.0920
Epoch 6/100
51573/51573 [==============================] - 18s 354us/sample - loss: 3.3224 - accuracy: 0.0969- loss: 3.3227 - accuracy
Epoch 7/100
51573/51573 [==============================] - 19s 368us/sample - loss: 3.2819 - accuracy: 0.1068
Epoch 8/100
51573/51573 [==============================] - 18s 351us/sample - loss: 3.2411 - accuracy: 0.1136
Epoch 9/100
51573/51573 [==============================] - 18s 351us/sam

In [270]:
print(makeModel.evaluate(X_make_test,y_make_test,verbose=0))
print(makeModel.evaluate(X_make_test,y_make_test,verbose=0))
print(makeModel.evaluate(X_make_test,y_make_test,verbose=0))

[1.6856130433670584, 0.5954708]
[1.6856130433670584, 0.5954708]
[1.6856130433670584, 0.5954708]


In [271]:
batch_size = 50  # 每批训练数据量的大小
epochs = 50
makeModel.fit(X_make_train,
              y_make_train,
              epochs=epochs,
              batch_size=batch_size,
              shuffle=True,
              #                    validation_data=(valid_input, valid_output)
              )

Train on 51573 samples
Epoch 1/50
51573/51573 [==============================] - 19s 374us/sample - loss: 1.2928 - accuracy: 0.6220
Epoch 2/50
51573/51573 [==============================] - 20s 396us/sample - loss: 1.2824 - accuracy: 0.6217
Epoch 3/50
51573/51573 [==============================] - 19s 375us/sample - loss: 1.2916 - accuracy: 0.6192
Epoch 4/50
51573/51573 [==============================] - 19s 370us/sample - loss: 1.2823 - accuracy: 0.6253
Epoch 5/50
51573/51573 [==============================] - 19s 372us/sample - loss: 1.2783 - accuracy: 0.6258
Epoch 6/50
51573/51573 [==============================] - 19s 373us/sample - loss: 1.2739 - accuracy: 0.6275
Epoch 7/50
51573/51573 [==============================] - 19s 370us/sample - loss: 1.2720 - accuracy: 0.6283
Epoch 8/50
51573/51573 [==============================] - 20s 380us/sample - loss: 1.2616 - accuracy: 0.6297
Epoch 9/50
51573/51573 [==============================] - 19s 375us/sample - loss: 1.2631 - accuracy: 0.6

KeyboardInterrupt: 

In [272]:
print(makeModel.evaluate(X_make_test,y_make_test,verbose=0))

[1.6840786233102885, 0.604157]


In [273]:
batch_size = 60  # 每批训练数据量的大小
epochs = 50
makeModel.fit(X_make_train,
              y_make_train,
              epochs=epochs,
              batch_size=batch_size,
              shuffle=True,
              #                    validation_data=(valid_input, valid_output)
              )
print(makeModel.evaluate(X_make_test,y_make_test,verbose=0))

Train on 51573 samples
Epoch 1/50
51573/51573 [==============================] - 17s 331us/sample - loss: 1.1379 - accuracy: 0.6632- loss: 1.1379 - accuracy: 0.66
Epoch 2/50
51573/51573 [==============================] - 17s 325us/sample - loss: 1.1163 - accuracy: 0.6691
Epoch 3/50
51573/51573 [==============================] - 19s 359us/sample - loss: 1.1233 - accuracy: 0.6656
Epoch 4/50
51573/51573 [==============================] - 18s 345us/sample - loss: 1.1159 - accuracy: 0.6686
Epoch 5/50
51573/51573 [==============================] - 18s 348us/sample - loss: 1.1222 - accuracy: 0.6663
Epoch 6/50
51573/51573 [==============================] - 18s 347us/sample - loss: 1.1153 - accuracy: 0.6687
Epoch 7/50
51573/51573 [==============================] - 18s 348us/sample - loss: 1.0984 - accuracy: 0.6729
Epoch 8/50
51573/51573 [==============================] - 18s 348us/sample - loss: 1.1080 - accuracy: 0.6720
Epoch 9/50
51573/51573 [==============================] - 18s 347us/sample

In [277]:
batch_size = 64  # 每批训练数据量的大小
epochs = 50
makeModel.fit(X_make_train,
              y_make_train,
              epochs=epochs,
              batch_size=batch_size,
              shuffle=True,
              #                    validation_data=(valid_input, valid_output)
              )
print(makeModel.evaluate(X_make_test,y_make_test,verbose=0))

Train on 51573 samples
Epoch 1/50
51573/51573 [==============================] - 17s 320us/sample - loss: 0.9981 - accuracy: 0.7024
Epoch 2/50
51573/51573 [==============================] - 16s 319us/sample - loss: 1.0050 - accuracy: 0.6992
Epoch 3/50
51573/51573 [==============================] - 16s 319us/sample - loss: 0.9952 - accuracy: 0.7027- loss: 0.9
Epoch 4/50
51573/51573 [==============================] - 16s 318us/sample - loss: 0.9907 - accuracy: 0.7038
Epoch 5/50
51573/51573 [==============================] - 17s 323us/sample - loss: 0.9939 - accuracy: 0.7015
Epoch 6/50
51573/51573 [==============================] - 23s 443us/sample - loss: 0.9983 - accuracy: 0.7018
Epoch 7/50
51573/51573 [==============================] - 23s 447us/sample - loss: 0.9909 - accuracy: 0.7035
Epoch 8/50
51573/51573 [==============================] - 23s 442us/sample - loss: 0.9848 - accuracy: 0.7059
Epoch 9/50
51573/51573 [==============================] - 23s 437us/sample - loss: 0.9917 - ac

In [ ]:
# MODEL_PATH = 'model/make_model.h5'
# bodyStyleModel.save(MODEL_PATH)